$$
\def\CC{\bf C}
\def\QQ{\bf Q}
\def\RR{\bf R}
\def\ZZ{\bf Z}
\def\NN{\bf N}
$$
# Abelian Sandpile Model

David Perkinson, Reed College

## Introduction

These notes provide an introduction to Dhar's abelian sandpile model
(ASM) and to Sage Sandpiles, a collection of tools in Sage for doing
sandpile calculations. For a more thorough introduction to the theory of
the ASM, the papers *Chip-Firing and Rotor-Routing on Directed Graphs*
<a href="#H" class="citation">[H]</a> by Holroyd et al., and
*Riemann-Roch and Abel-Jacobi Theory on a Finite Graph* by Baker and
Norine <a href="#BN" class="citation">[BN]</a> are recommended. See also
<a href="#PPW2013" class="citation">[PPW2013]</a>.

To describe the ASM, we start with a *sandpile graph*: a directed
multigraph $\Gamma$ with a vertex $s$ that is accessible from every
vertex. By *multigraph*, we mean that each edge of $\Gamma$ is assigned
a nonnegative integer weight. To say $s$ is *accessible* from some
vertex $v$ means that there is a sequence (possibly empty) of directed
edges starting at $v$ and ending at $s$. We call $s$ the *sink* of the
sandpile graph, even though it might have outgoing edges, for reasons
that will be made clear in a moment.

We denote the vertex set of $\Gamma$ by $V$, and define
$\tilde{V} = V\setminus\{s\}$. For any vertex $v$, we let
$\mbox{out-degree}(v)$ (the *out-degree* of $v$) be the sum of the
weights of all edges leaving $v$.

### Configurations and divisors

A *configuration* on $\Gamma$ is an element of $\ZZ_{\geq 0} \tilde{V}$,
i.e., the assignment of a nonnegative integer to each nonsink vertex. We
think of each integer as a number of grains of sand being placed at the
corresponding vertex. A *divisor* on $\Gamma$ is an element of $\ZZ V$,
i.e., an element in the free abelian group on *all* of the vertices. In
the context of divisors, it is sometimes useful to think of a divisor on
$\Gamma$ as assigning dollars to each vertex, with negative integers
signifying a debt.

### Stabilization

A configuration $c$ is *stable* at a vertex $v\in\tilde{V}$ if
$c(v) < \mbox{out-degree}(v)$. Otherwise, $c$ is *unstable* at $v$. A
configuration $c$ (in itself) is *stable* if it is stable at each
nonsink vertex. Otherwise, $c$ is *unstable*. If $c$ is unstable at $v$,
the vertex $v$ can be *fired* (*toppled*) by removing
$\mbox{out-degree}(v)$ grains of sand from $v$ and adding grains of sand
to the neighbors of $v$, determined by the weights of the edges leaving
$v$ (each vertex $w$ gets as many grains as the weight of the edge
$(v, w)$ is, if there is such an edge). Note that grains that are added
to the sink $s$ are not counted (i.e., they "disappear").

Despite our best intentions, we sometimes consider firing a stable
vertex, resulting in a "configuration" with a negative amount of sand at
that vertex. We may also *reverse-fire* a vertex, absorbing sand from
the vertex's neighbors (including the sink).

**Example.** Consider the graph:

<figure>
<img src="media/sandpile/example1.png" class="align-center" alt="" /><figcaption><span class="math inline"><em>Γ</em></span></figcaption>
</figure>

All edges have weight $1$ except for the edge from vertex 1 to vertex 3,
which has weight $2$. If we let $c = (5,0,1)$ with the indicated number
of grains of sand on vertices 1, 2, and 3, respectively, then only
vertex 1, whose out-degree is 4, is unstable. Firing vertex 1 gives a
new configuration $c' = (1,1,3)$. Here, $4$ grains have left vertex 1.
One of these has gone to the sink vertex (and forgotten), one has gone
to vertex 2, and two have gone to vertex 3, since the edge from 1 to 3
has weight $2$. Vertex 3 in the new configuration is now unstable. The
Sage code for this example follows. :

In [ ]:
g = {0:{},
     1:{0:1, 2:1, 3:2},
     2:{1:1, 3:1},
     3:{1:1, 2:1}}
S = Sandpile(g, 0)   # create the sandpile
S.show(edge_labels=true)  # display the graph

Create the configuration:

In [ ]:
c = SandpileConfig(S, {1:5, 2:0, 3:1})
S.out_degree()

{0: 0, 1: 4, 2: 2, 3: 2}

Fire vertex $1$ :

In [ ]:
c.fire_vertex(1)

{1: 1, 2: 1, 3: 3}

The configuration is unchanged:

In [ ]:
c

{1: 5, 2: 0, 3: 1}

Repeatedly fire vertices until the configuration becomes stable:

In [ ]:
c.stabilize()

{1: 2, 2: 1, 3: 1}

Alternatives:

In [ ]:
~c             # shorthand for c.stabilize()

{1: 2, 2: 1, 3: 1}

In [ ]:
c.stabilize(with_firing_vector=true)

[{1: 2, 2: 1, 3: 1}, {1: 2, 2: 2, 3: 3}]

Since vertex 3 has become unstable after firing vertex 1, it can be
fired, which causes vertex 2 to become unstable, etc. Repeated firings
eventually lead to a stable configuration. The last line of the Sage
code, above, is a list, the first element of which is the resulting
stable configuration, $(2,1,1)$. The second component records how many
times each vertex fired in the stabilization.

------------------------------------------------------------------------

Since the sink is accessible from each nonsink vertex and never fires,
every configuration will stabilize after a finite number of
vertex-firings. It is not obvious, but the resulting stabilization is
independent of the order in which unstable vertices are fired. Thus,
each configuration stabilizes to a unique stable configuration.

### Laplacian

Fix a total order on the vertices of $\Gamma$, thus leading to a
labelling of the vertices by the numbers $1, 2, \ldots, n$ for some $n$
(in the given order). The *Laplacian* of $\Gamma$ is

$$L := D-A$$

where $D$ is the diagonal matrix of out-degrees of the vertices (i.e.,
the diagonal $n \times n$-matrix whose $(i, i)$-th entry is the
out-degree of the vertex $i$) and $A$ is the adjacency matrix whose
$(i,j)$-th entry is the weight of the edge from vertex $i$ to vertex
$j$, which we take to be $0$ if there is no edge. The *reduced
Laplacian*, $\tilde{L}$, is the submatrix of the Laplacian formed by
removing the row and column corresponding to the sink vertex. Firing a
vertex of a configuration is the same as subtracting the corresponding
row of the reduced Laplacian.

**Example.** (Continued.) :

In [ ]:
S.vertices(sort=True)  # the ordering of the vertices

[0, 1, 2, 3]

In [ ]:
S.laplacian()

[ 0  0  0  0]
[-1  4 -1 -2]
[ 0 -1  2 -1]
[ 0 -1 -1  2]

In [ ]:
S.reduced_laplacian()

[ 4 -1 -2]
[-1  2 -1]
[-1 -1  2]

The configuration we considered previously:

In [ ]:
c = SandpileConfig(S, [5,0,1])
c

{1: 5, 2: 0, 3: 1}

Firing vertex 1 is the same as subtracting the corresponding row of the
reduced Laplacian from the configuration (regarded as a vector):

In [ ]:
c.fire_vertex(1).values()

[1, 1, 3]

In [ ]:
S.reduced_laplacian()[0]

(4, -1, -2)

In [ ]:
vector([5,0,1]) - vector([4,-1,-2])

(1, 1, 3)

### Recurrent elements

Imagine an experiment in which grains of sand are dropped one-at-a-time
onto a graph, pausing to allow the configuration to stabilize between
drops. Some configurations will only be seen once in this process. For
example, for most graphs, once sand is dropped on the graph, no sequence
of additions of sand and stabilizations will result in a graph empty of
sand. Other configurations---the so-called *recurrent
configurations*---will be seen infinitely often as the process is
repeated indefinitely.

To be precise, a configuration $c$ is *recurrent* if (i) it is stable,
and (ii) given any configuration $a$, there is a configuration $b$ such
that $c = \mbox{stab}(a+b)$, the stabilization of $a + b$.

The *maximal-stable* configuration, denoted $c_{\mathrm{max}}$, is
defined by $c_{\mathrm{max}}(v)=\mbox{out-degree}(v)-1$ for all nonsink
vertices $v$. It is clear that $c_{\mathrm{max}}$ is recurrent. Further,
it is not hard to see that a configuration is recurrent if and only if
it has the form $\mbox{stab}(a+c_{\mathrm{max}})$ for some configuration
$a$.

**Example.** (Continued.) :

In [ ]:
S.recurrents(verbose=false)

[[3, 1, 1], [2, 1, 1], [3, 1, 0]]

In [ ]:
c = SandpileConfig(S, [2,1,1])
c

{1: 2, 2: 1, 3: 1}

In [ ]:
c.is_recurrent()

True

In [ ]:
S.max_stable()

{1: 3, 2: 1, 3: 1}

Adding any configuration to the max-stable configuration and stabilizing
yields a recurrent configuration:

In [ ]:
x = SandpileConfig(S, [1,0,0])
x + S.max_stable()

{1: 4, 2: 1, 3: 1}

Use `&` to add and stabilize:

In [ ]:
c = x & S.max_stable()
c

{1: 3, 2: 1, 3: 0}

In [ ]:
c.is_recurrent()

True

Note the various ways of performing addition and stabilization:

In [ ]:
m = S.max_stable()
(x + m).stabilize() == ~(x + m)

True

In [ ]:
(x + m).stabilize() == x & m

True

### Burning Configuration

A *burning configuration* is a $\ZZ_{\geq 0}$-linear combination $f$ of
the rows of the reduced Laplacian matrix having nonnegative entries and
such that every vertex is accessible from some vertex in the support of
$f$ (in other words, for each vertex $w$, there is a path from some
$v \in \operatorname{supp} f$ to $w$). The corresponding *burning
script* gives the coefficients in the $\ZZ_{\geq 0}$-linear combination
needed to obtain the burning configuration. So if $b$ is a burning
configuration, $\sigma$ is its script, and $\tilde{L}$ is the reduced
Laplacian, then $\sigma\,\tilde{L} = b$. The *minimal burning
configuration* is the one with the minimal script (each of its
components is no larger than the corresponding component of any other
script for a burning configuration).

Given a burning configuration $b$ having script $\sigma$, and any
configuration $c$ on the same graph, the following are equivalent:

-   $c$ is recurrent;
-   $c + b$ stabilizes to $c$;
-   the firing vector for the stabilization of $c + b$ is $\sigma$.

The burning configuration and script are computed using a modified
version of Speer's script algorithm. This is a generalization to
directed multigraphs of Dhar's burning algorithm.

**Example.** :

In [ ]:
g = {0:{},1:{0:1,3:1,4:1},2:{0:1,3:1,5:1},
     3:{2:1,5:1},4:{1:1,3:1},5:{2:1,3:1}}
G = Sandpile(g,0)
G.burning_config()

{1: 2, 2: 0, 3: 1, 4: 1, 5: 0}

In [ ]:
G.burning_config().values()

[2, 0, 1, 1, 0]

In [ ]:
G.burning_script()

{1: 1, 2: 3, 3: 5, 4: 1, 5: 4}

In [ ]:
G.burning_script().values()

[1, 3, 5, 1, 4]

In [ ]:
matrix(G.burning_script().values())*G.reduced_laplacian()

[2 0 1 1 0]

### Sandpile group

The collection of stable configurations forms a commutative monoid with
addition defined as ordinary addition followed by stabilization. The
identity element is the all-zero configuration. This monoid is a group
exactly when the underlying graph is a DAG (directed acyclic graph).

The recurrent elements form a submonoid which turns out to be a group.
This group is called the *sandpile group* for $\Gamma$, denoted
$\mathcal{S}(\Gamma)$. Its identity element is usually not the all-zero
configuration (again, only in the case that $\Gamma$ is a DAG). So
finding the identity element is an interesting problem.

Let $n=|V|-1$ and fix an ordering of the nonsink vertices. Let
$\mathcal{\tilde{L}}\subset\ZZ^n$ denote the column-span of
$\tilde{L}^t$, the transpose of the reduced Laplacian. It is a theorem
that

$$\mathcal{S}(\Gamma)\approx \ZZ^n/\mathcal{\tilde{L}}.$$

Thus, the number of elements of the sandpile group is $\det{\tilde{L}}$,
which by the matrix-tree theorem is the number of weighted trees
directed into the sink.

**Example.** (Continued.) :

In [ ]:
S.group_order()

3

In [ ]:
S.invariant_factors()

[1, 1, 3]

In [ ]:
S.reduced_laplacian().dense_matrix().smith_form()

(
[1 0 0]  [ 1  0  0]  [1 3 5]
[0 1 0]  [ 0  1  0]  [1 4 6]
[0 0 3], [ 0 -1  1], [1 4 7]
)

Adding the identity to any recurrent configuration and stabilizing
yields the same recurrent configuration:

In [ ]:
S.identity()

{1: 3, 2: 1, 3: 0}

In [ ]:
i = S.identity()
m = S.max_stable()
i & m == m

True

### Self-organized criticality

The sandpile model was introduced by Bak, Tang, and Wiesenfeld in the
paper, *Self-organized criticality: an explanation of 1/ƒ noise*
<a href="#BTW" class="citation">[BTW]</a>. The term *self-organized
criticality* has no precise definition, but can be loosely taken to
describe a system that naturally evolves to a state that is barely
stable and such that the instabilities are described by a power law. In
practice, *self-organized criticality* is often taken to mean *like the
sandpile model on a grid graph*. The grid graph is just a grid with an
extra sink vertex. The vertices on the interior of each side have one
edge to the sink, and the corner vertices have an edge of weight $2$.
Thus, every nonsink vertex has out-degree $4$.

Imagine repeatedly dropping grains of sand on an empty grid graph,
allowing the sandpile to stabilize in between. At first there is little
activity, but as time goes on, the size and extent of the avalanche
caused by a single grain of sand becomes hard to predict. Computer
experiments---I do not think there is a proof, yet---indicate that the
distribution of avalanche sizes obeys a power law with exponent -1. In
the example below, the size of an avalanche is taken to be the sum of
the number of times each vertex fires.

**Example (distribution of avalanche sizes).** :

In [ ]:
S = sandpiles.Grid(10,10)
m = S.max_stable()
a = []
for i in range(10000):  # long time (15s on sage.math, 2012)
    m = m.add_random()
    m, f = m.stabilize(true)
    a.append(sum(f.values()))
p = list_plot([[log(i+1),log(a.count(i))] for i in [0..max(a)] if a.count(i)])  # long time
p.axes_labels(['log(N)','log(D(N))'])  # long time
p  # long time

Graphics object consisting of 1 graphics primitive

<figure>
<img src="media/sandpile/btw.png" class="align-center" alt="" /><figcaption>Distribution of avalanche sizes</figcaption>
</figure>

Note: In the above code, `m.stabilize(true)` returns a list consisting
of the stabilized configuration and the firing vector. (Omitting `true`
would give just the stabilized configuration.)

### Divisors and Discrete Riemann surfaces

A reference for this section is *Riemann-Roch and Abel-Jacobi theory on
a finite graph* <a href="#BN" class="citation">[BN]</a>.

A *divisor* on $\Gamma$ is an element of the free abelian group on its
vertices, including the sink. Suppose, as above, that the $n+1$ vertices
of $\Gamma$ have been ordered, and that $\mathcal{L}$ is the column span
of the transpose of the Laplacian. A divisor is then identified with an
element $D\in\ZZ^{n+1}$, and two divisors are *linearly equivalent* if
they differ by an element of $\mathcal{L}$. A divisor $E$ is
*effective*, written $E\geq0$, if $E(v)\geq0$ for each $v\in V$, i.e.,
if $E\in\mathbb{Z}_{\geq 0}^{n+1}$. The *degree* of a divisor $D$ is
$\deg(D) := \sum_{v\in V} D(v)$. The divisors of degree zero modulo
linear equivalence form the *Picard group*, or *Jacobian* of the graph.
For an undirected graph, the Picard group is isomorphic to the sandpile
group.

The *complete linear system* for a divisor $D$, denoted $|D|$, is the
collection of effective divisors linearly equivalent to $D$.

#### Riemann-Roch

To describe the Riemann-Roch theorem in this context, suppose that
$\Gamma$ is an undirected, unweighted graph. The *dimension*, $r(D)$, of
the linear system $|D|$ is $-1$ if $|D|=\emptyset$, and otherwise is the
greatest integer $s$ such that $|D-E|\neq0$ for all effective divisors
$E$ of degree $s$. Define the *canonical divisor* by
$K = \sum_{v\in V} (\deg(v)-2) v$, and the *genus* by
$g = \#(E) - \#(V) + 1$. The Riemann-Roch theorem says that for any
divisor $D$,

$$r(D) - r(K-D) = \deg(D) + 1 - g.$$

**Example.** :

In [ ]:
G = sandpiles.Complete(5)  # the sandpile on the complete graph with 5 vertices

A divisor on the graph:

In [ ]:
D = SandpileDivisor(G, [1,2,2,0,2])

Verify the Riemann-Roch theorem:

In [ ]:
K = G.canonical_divisor()
D.rank() - (K - D).rank() == D.deg() + 1 - G.genus()

True

The effective divisors linearly equivalent to $D$ :

In [ ]:
D.effective_div(False)

[[0, 1, 1, 4, 1], [1, 2, 2, 0, 2], [4, 0, 0, 3, 0]]

The nonspecial divisors up to linear equivalence (divisors of degree
$g-1$ with empty linear systems):

In [ ]:
N = G.nonspecial_divisors()
[E.values() for E in N[:5]]   # the first few

[[-1, 0, 1, 2, 3],
 [-1, 0, 1, 3, 2],
 [-1, 0, 2, 1, 3],
 [-1, 0, 2, 3, 1],
 [-1, 0, 3, 1, 2]]

In [ ]:
len(N)

24

In [ ]:
len(N) == G.h_vector()[-1]

True

#### Picturing linear systems

Fix a divisor $D$. There are at least two natural graphs associated with
the linear system associated with $D$. First, consider the directed
graph with vertex set $|D|$ and with an edge from vertex $E$ to vertex
$F$ if $F$ is attained from $E$ by firing a single unstable vertex. :

In [ ]:
S = Sandpile(graphs.CycleGraph(6),0)
D = SandpileDivisor(S, [1,1,1,1,2,0])
D.is_alive()

True

In [ ]:
eff = D.effective_div()
firing_graph(S,eff).show3d(edge_size=.005,vertex_size=0.01,iterations=500)

<figure>
<img src="media/sandpile/C_6.png" class="align-center" alt="" /><figcaption>Complete linear system for <span class="math inline">(1, 1, 1, 1, 2, 0)</span> on <span class="math inline"><em>C</em><sub>6</sub></span> : single firings</figcaption>
</figure>

The `is_alive` method checks whether the divisor $D$ is alive, i.e.,
whether every divisor linearly equivalent to $D$ is unstable.

The second graph has the same set of vertices but with an edge from $E$
to $F$ if $F$ is obtained from $E$ by firing all unstable vertices of
$E$. :

In [ ]:
S = Sandpile(graphs.CycleGraph(6),0)
D = SandpileDivisor(S, [1,1,1,1,2,0])
eff = D.effective_div() 
parallel_firing_graph(S,eff).show3d(edge_size=.005,vertex_size=0.01,iterations=500)

<figure>
<img src="media/sandpile/C_6-parallel.png" class="align-center" alt="" /><figcaption>Complete linear system for <span class="math inline">(1, 1, 1, 1, 2, 0)</span> on <span class="math inline"><em>C</em><sub>6</sub></span> : parallel firings</figcaption>
</figure>

Note that in each of the examples above, starting at any divisor in the
linear system and following edges, one is eventually led into a cycle of
length $6$ (cycling the divisor $(1,1,1,1,2,0)$). Thus, `D.alive()`
returns `True`. In Sage, one would be able to rotate the above figures
to get a better idea of the structure.

### Algebraic geometry of sandpiles

#### Affine

Let $n = |V| - 1$, and fix an ordering on the nonsink vertices of
$\Gamma$. Let $\tilde{\mathcal{L}} \subset \ZZ^n$ denote the column-span
of $\tilde{L}^t$, the transpose of the reduced Laplacian. Label vertex
$i$ with the indeterminate $x_i$, and let
$\CC[\Gamma_s] = \CC[x_1, \dots, x_n]$. (Here, $s$ denotes the sink
vertex of $\Gamma$.) The *sandpile ideal* or *toppling ideal*, first
studied by Cori, Rossin, and Salvy
<a href="#CRS" class="citation">[CRS]</a> for undirected graphs, is the
lattice ideal for $\tilde{\mathcal{L}}$ :

$$I = I(\Gamma_s) := \CC[\Gamma_s] \cdot \{x^u-x^v : u-v \in
\tilde{\mathcal{L}}\}\subset\CC[\Gamma_s],$$

where $x^u := \prod_{i=1}^n x^{u_i}$ for $u \in \ZZ^n$.

For each $c\in\ZZ^n$ define $t(c) = x^{c^+} - x^{c^-}$ where
$c^+_i = \max\{c_i,0\}$ and $c^-_i = \max\{-c_i,0\}$, so that
$c = c^+ - c^-$. Then, for each $\sigma \in \ZZ^n$, define $T(\sigma) =
t(\tilde{L}^t\sigma)$. It then turns out that

$$I = (T(e_1),\dots,T(e_n),x^b-1),$$

where $e_i$ is the $i$-th standard basis vector and $b$ is any burning
configuration.

The affine coordinate ring, $\CC[\Gamma_s]/I,$ is isomorphic to the
group algebra of the sandpile group, $\CC[\mathcal{S}(\Gamma)]$.

The standard term-ordering on $\CC[\Gamma_s]$ is graded reverse
lexicographical order with $x_i > x_j$ if vertex $v_i$ is further from
the sink than vertex $v_j$. (There are choices to be made for vertices
equidistant from the sink.) If $\sigma_b$ is the script for a burning
configuration (not necessarily minimal), then

$$\{T(\sigma) : \sigma \leq \sigma_b\}$$

is a Gröbner basis for $I$.

#### Projective

Now let $\CC[\Gamma] = \CC[x_0,x_1,\dots,x_n]$, where $x_0$ corresponds
to the sink vertex. The *homogeneous sandpile ideal*, denoted $I^h$, is
obtaining by homogenizing $I$ with respect to $x_0$. Let $L$ be the
(full) Laplacian, and $\mathcal{L} \subset \ZZ^{n+1}$ be the column span
of its transpose, $L^t.$ Then $I^h$ is the lattice ideal for
$\mathcal{L}$ :

$$I^h = I^h(\Gamma) := \CC[\Gamma] \cdot \{x^u-x^v: u-v \in\mathcal{L}\}
\subset \CC[\Gamma].$$

This ideal can be calculated by saturating the ideal

$$(T(e_i): i=0, \ldots, n)$$

with respect to the product of the indeterminates: $\prod_{i=0}^n x_i$
(extending the $T$ operator in the obvious way). A Gröbner basis with
respect to the degree lexicographic order described above (with $x_0$
the smallest vertex) is obtained by homogenizing each element of the
Gröbner basis for the non-homogeneous sandpile ideal with respect to
$x_0$.

**Example.** :

In [ ]:
g = {0:{},1:{0:1,3:1,4:1},2:{0:1,3:1,5:1},
     3:{2:1,5:1},4:{1:1,3:1},5:{2:1,3:1}}
S = Sandpile(g, 0)
S.ring()

Multivariate Polynomial Ring in x5, x4, x3, x2, x1, x0 over Rational Field

The homogeneous sandpile ideal:

In [ ]:
S.ideal()

Ideal (x2 - x0, x3^2 - x5*x0, x5*x3 - x0^2, x4^2 - x3*x1, x5^2 - x3*x0,
x1^3 - x4*x3*x0, x4*x1^2 - x5*x0^2) of Multivariate Polynomial Ring
in x5, x4, x3, x2, x1, x0 over Rational Field

The generators of the ideal:

In [ ]:
S.ideal(true)

[x2 - x0,
 x3^2 - x5*x0,
 x5*x3 - x0^2,
 x4^2 - x3*x1,
 x5^2 - x3*x0,
 x1^3 - x4*x3*x0,
 x4*x1^2 - x5*x0^2]

Its resolution:

In [ ]:
S.resolution() # long time

'R^1 <-- R^7 <-- R^19 <-- R^25 <-- R^16 <-- R^4'

and Betti table:

In [ ]:
S.betti() # long time

           0     1     2     3     4     5
------------------------------------------
    0:     1     1     -     -     -     -
    1:     -     4     6     2     -     -
    2:     -     2     7     7     2     -
    3:     -     -     6    16    14     4
------------------------------------------
total:     1     7    19    25    16     4

The Hilbert function:

In [ ]:
S.hilbert_function()

[1, 5, 11, 15]

and its first differences (which count the number of superstable
configurations in each degree):

In [ ]:
S.h_vector()

[1, 4, 6, 4]

In [ ]:
x = [i.deg() for i in S.superstables()]
sorted(x)

[0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3]

The degree in which the Hilbert function starts equalling the Hilbert
polynomial, the latter always being a constant in the case of a sandpile
ideal:

In [ ]:
S.postulation()

3

#### Zeros

The *zero set* for the sandpile ideal $I$ is

$$Z(I) = \{ p\in\CC^n : f(p)=0\mbox{ for all $f\in I$} \},$$

the set of simultaneous zeros of the polynomials in $I$. Letting $S^1$
denote the unit circle in the complex plane, $Z(I)$ is a finite subgroup
of $S^1 \times \cdots \times S^1 \subset \CC^n$, isomorphic to the
sandpile group. The zero set is actually linearly isomorphic to a
faithful representation of the sandpile group on $\CC^n$.

The above is not quite true. $Z(I)$ is neither finite nor a subgroup of
$S^1 \times \cdots \times S^1 \subset \CC^n$. What is probably meant is
that the subset of $Z(I)$ in which the coordinate of $p$ corresponding
to the sink is set to $1$ is a finite subgroup of
$S^1 \times \cdots \times S^1 \subset \CC^{n-1}$.

**Example.** (Continued.) :

In [ ]:
S = Sandpile({0: {}, 1: {2: 2}, 2: {0: 4, 1: 1}}, 0)
S.ideal().gens()

[x1^2 - x2^2, x1*x2^3 - x0^4, x2^5 - x1*x0^4]

Approximation to the zero set (setting `x_0 = 1`):

In [ ]:
S.solve()

[[-0.707107000000000 + 0.707107000000000*I,
  0.707107000000000 - 0.707107000000000*I],
 [-0.707107000000000 - 0.707107000000000*I,
  0.707107000000000 + 0.707107000000000*I],
 [-I, -I],
 [I, I],
 [0.707107000000000 + 0.707107000000000*I,
  -0.707107000000000 - 0.707107000000000*I],
 [0.707107000000000 - 0.707107000000000*I,
  -0.707107000000000 + 0.707107000000000*I],
 [1, 1],
 [-1, -1]]

In [ ]:
len(_) == S.group_order()

True

The zeros are generated as a group by a single vector:

In [ ]:
S.points()

[[-(1/2*I + 1/2)*sqrt(2), (1/2*I + 1/2)*sqrt(2)]]

#### Resolutions

The homogeneous sandpile ideal, $I^h$, has a free resolution graded by
the divisors on $\Gamma$ modulo linear equivalence. (See the section on
[Discrete Riemann Surfaces](discrete_riemann_surfaces.ipynb) for the
language of divisors and linear equivalence.) Let
$S = \CC[\Gamma] = \CC[x_0,\dots,x_n]$, as above, and let $\mathfrak{S}$
denote the group of divisors modulo rational equivalence. Then $S$ is
graded by $\mathfrak{S}$ by letting $\deg(x^c) = c \in \mathfrak{S}$ for
each monomial $x^c$. The minimal free resolution of $I^h$ has the form

$$0 \leftarrow I^h
\leftarrow \bigoplus_{D\in\mathfrak{S}} S(-D)^{\beta_{0,D}}
\leftarrow \bigoplus_{D\in\mathfrak{S}} S(-D)^{\beta_{1,D}}
\leftarrow \cdots
\leftarrow \bigoplus_{D\in\mathfrak{S}} S(-D)^{\beta_{r,D}}
\leftarrow 0,$$

where the $\beta_{i,D}$ are the *Betti numbers* for $I^h$.

For each divisor class $D\in\mathfrak{S}$, define a simplicial complex

$$\Delta_D := \{I\subseteq\{0,\dots,n\}: I\subseteq\mbox{supp}(E)
\mbox{ for some } E \in |D|\}.$$

The Betti number $\beta_{i,D}$ equals the dimension over $\CC$ of the
$i$-th reduced homology group of $\Delta_D$ :

$$\beta_{i,D} = \dim_{\CC}\tilde{H}_i(\Delta_D;\CC).$$

In [ ]:
S = Sandpile({0:{},1:{0: 1, 2: 1, 3: 4},2:{3: 5},3:{1: 1, 2: 1}},0)

Representatives of all divisor classes with nontrivial homology:

In [ ]:
p = S.betti_complexes()
p[0]

[{0: -8, 1: 5, 2: 4, 3: 1},
 Simplicial complex with vertex set (1, 2, 3) and facets {(3,), (1, 2)}]

The homology associated with the first divisor in the list:

In [ ]:
D = p[0][0]
D.effective_div()

[{0: 0, 1: 0, 2: 0, 3: 2}, {0: 0, 1: 1, 2: 1, 3: 0}]

In [ ]:
[E.support() for E in D.effective_div()]

[[3], [1, 2]]

In [ ]:
D.Dcomplex()

Simplicial complex with vertex set (1, 2, 3) and facets {(3,), (1, 2)}

In [ ]:
D.Dcomplex().homology()

{0: Z, 1: 0}

The minimal free resolution:

In [ ]:
S.resolution()

'R^1 <-- R^5 <-- R^5 <-- R^1'

In [ ]:
S.betti()

           0     1     2     3
------------------------------
    0:     1     -     -     -
    1:     -     5     5     -
    2:     -     -     -     1
------------------------------
total:     1     5     5     1

In [ ]:
len(p)

11

The degrees and ranks of the homology groups for each element of the
list `p` (compare with the Betti table, above):

In [ ]:
[[sum(d[0].values()),d[1].betti()] for d in p]

[[2, {0: 2, 1: 0}],
 [3, {0: 1, 1: 1, 2: 0}],
 [2, {0: 2, 1: 0}],
 [3, {0: 1, 1: 1, 2: 0}],
 [2, {0: 2, 1: 0}],
 [3, {0: 1, 1: 1, 2: 0}],
 [2, {0: 2, 1: 0}],
 [3, {0: 1, 1: 1}],
 [2, {0: 2, 1: 0}],
 [3, {0: 1, 1: 1, 2: 0}],
 [5, {0: 1, 1: 0, 2: 1}]]

#### Complete Intersections and Arithmetically Gorenstein toppling ideals

NOTE: in the previous section note that the resolution always has length
$n$ since the ideal is Cohen-Macaulay.

To do.

#### Betti numbers for undirected graphs

To do.

## Usage

### Initialization

There are three main classes for sandpile structures in Sage:
`Sandpile`, `SandpileConfig`, and `SandpileDivisor`. Initialization for
`Sandpile` has the form :

In [ ]:
.. skip

> sage: S = Sandpile(graph, sink)

where `graph` represents a graph and `sink` is the key for the sink
vertex. There are four possible forms for `graph` :

1.  a Python dictionary of dictionaries:

In [ ]:
    g = {0: {}, 1: {0: 1, 3: 1, 4: 1}, 2: {0: 1, 3: 1, 5: 1},
         3: {2: 1, 5: 1}, 4: {1: 1, 3: 1}, 5: {2: 1, 3: 1}}


    <figure>
    <img src="media/sandpile/initial.png" class="align-center" alt="" />
    </figure>

    Graph from dictionary of dictionaries.

    Each key is the name of a vertex. Next to each vertex name $v$ is a
    dictionary consisting of pairs: `vertex: weight`. Each pair
    represents a directed edge emanating from $v$ and ending at `vertex`
    having (non-negative integer) weight equal to `weight`. Loops are
    allowed. In the example above, all of the weights are 1.


2.  a Python dictionary of lists:

In [ ]:
    g = {0: [], 1: [0, 3, 4], 2: [0, 3, 5],
         3: [2, 5], 4: [1, 3], 5: [2, 3]}


    This is a short-hand when all of the edge-weights are equal to 1.
    The above example is for the same displayed graph.


3.  a Sage graph (of type `sage.graphs.graph.Graph`):

In [ ]:
    g = graphs.CycleGraph(5)
    S = Sandpile(g, 0)
    type(g)

<class 'sage.graphs.graph.Graph'>


    To see the types of built-in graphs, type `graphs.`, including the
    period, and hit TAB.


4.  a Sage digraph:

In [ ]:
    S = Sandpile(digraphs.RandomDirectedGNC(6), 0)
    S.show()


    <figure>
    <img src="media/sandpile/random.png" class="align-center" alt="" /><figcaption>A random graph.</figcaption>
    </figure>

    See
    [sage.graphs.graph\_generators](../reference/sage/graphs/graph_generators.html)
    for more information on the Sage graph library and graph
    constructors.


Each of these four formats is preprocessed by the Sandpile class so
that, internally, the graph is represented by the dictionary of
dictionaries format first presented. This internal format is returned by
`dict()` :

In [ ]:
S = Sandpile({0:[], 1:[0, 3, 4], 2:[0, 3, 5], 3: [2, 5], 4: [1, 3], 5: [2, 3]},0)
S.dict()

{0: {},
 1: {0: 1, 3: 1, 4: 1},
 2: {0: 1, 3: 1, 5: 1},
 3: {2: 1, 5: 1},
 4: {1: 1, 3: 1},
 5: {2: 1, 3: 1}}

Note

The user is responsible for assuring that each vertex has a directed
path into the designated sink. If the sink has out-edges, these will be
ignored for the purposes of sandpile calculations (but not calculations
on divisors).

Code for checking whether a given vertex is a sink:

In [ ]:
S = Sandpile({0:[], 1:[0, 3, 4], 2:[0, 3, 5], 3: [2, 5], 4: [1, 3], 5: [2, 3]},0)
[S.distance(v,0) for v in S.vertices(sort=True)] # 0 is a sink

[0, 1, 1, 2, 2, 2]

In [ ]:
[S.distance(v,1) for v in S.vertices(sort=True)] # 1 is not a sink

[+Infinity, 0, +Infinity, +Infinity, 1, +Infinity]

### Methods

Here are summaries of `Sandpile`, `SandpileConfig`, and
`SandpileDivisor` methods (functions). Each summary is followed by a
list of complete descriptions of the methods. There are many more
methods available for a Sandpile, e.g., those inherited from the class
DiGraph. To see them all, enter `dir(Sandpile)` or type `Sandpile.`,
including the period, and hit TAB.

#### Sandpile

**Summary of methods.**

-   [all\_k\_config](all_k_config.ipynb) --- The constant configuration
    with all values set to k.
-   [all\_k\_div](all_k_div.ipynb) --- The divisor with all values set
    to k.
-   [avalanche\_polynomial](avalanche_polynomial.ipynb) --- The
    avalanche polynomial.
-   [betti](betti.ipynb) --- The Betti table for the homogeneous
    toppling ideal.
-   [betti\_complexes](betti_complexes.ipynb) --- The support-complexes
    with non-trivial homology.
-   [burning\_config](burning_config.ipynb) --- The minimal burning
    configuration.
-   [burning\_script](burning_script.ipynb) --- A script for the minimal
    burning configuration.
-   [canonical\_divisor](canonical_divisor.ipynb) --- The canonical
    divisor.
-   [dict](dict.ipynb) --- A dictionary of dictionaries representing a
    directed graph.
-   [genus](genus.ipynb) --- The genus: (\# non-loop edges) - (\#
    vertices) + 1.
-   [groebner](groebner.ipynb) --- A Groebner basis for the homogeneous
    toppling ideal.
-   [group\_gens](group_gens.ipynb) --- A minimal list of generators for
    the sandpile group.
-   [group\_order](group_order.ipynb) --- The size of the sandpile
    group.
-   [h\_vector](h_vector.ipynb) --- The number of superstable
    configurations in each degree.
-   [help](help.ipynb) --- List of Sandpile-specific methods (not
    inherited from Graph).
-   [hilbert\_function](hilbert_function.ipynb) --- The Hilbert function
    of the homogeneous toppling ideal.
-   [ideal](ideal.ipynb) --- The saturated homogeneous toppling ideal.
-   [identity](identity.ipynb) --- The identity configuration.
-   [in\_degree](in_degree.ipynb) --- The in-degree of a vertex or a
    list of all in-degrees.
-   [invariant\_factors](invariant_factors.ipynb) --- The invariant
    factors of the sandpile group.
-   [is\_undirected](is_undirected.ipynb) --- Is the underlying graph
    undirected?
-   [jacobian\_representatives](jacobian_representatives.ipynb) ---
    Representatives for the elements of the Jacobian group.
-   [laplacian](laplacian.ipynb) --- The Laplacian matrix of the graph.
-   [markov\_chain](markov_chain.ipynb) --- The sandpile Markov chain
    for configurations or divisors.
-   [max\_stable](max_stable.ipynb) --- The maximal stable
    configuration.
-   [max\_stable\_div](max_stable_div.ipynb) --- The maximal stable
    divisor.
-   [max\_superstables](max_superstables.ipynb) --- The maximal
    superstable configurations.
-   [min\_recurrents](min_recurrents.ipynb) --- The minimal recurrent
    elements.
-   [nonsink\_vertices](nonsink_vertices.ipynb) --- The nonsink
    vertices.
-   [nonspecial\_divisors](nonspecial_divisors.ipynb) --- The nonspecial
    divisors.
-   [out\_degree](out_degree.ipynb) --- The out-degree of a vertex or a
    list of all out-degrees.
-   [picard\_representatives](picard_representatives.ipynb) ---
    Representatives of the divisor classes of degree d in the Picard
    group.
-   [points](points.ipynb) --- Generators for the multiplicative group
    of zeros of the sandpile ideal.
-   [postulation](postulation.ipynb) --- The postulation number of the
    toppling ideal.
-   [recurrents](recurrents.ipynb) --- The recurrent configurations.
-   [reduced\_laplacian](reduced_laplacian.ipynb) --- The reduced
    Laplacian matrix of the graph.
-   [reorder\_vertices](reorder_vertices.ipynb) --- A copy of the
    sandpile with vertex names permuted.
-   [resolution](resolution.ipynb) --- A minimal free resolution of the
    homogeneous toppling ideal.
-   [ring](ring.ipynb) --- The ring containing the homogeneous toppling
    ideal.
-   [show](show.ipynb) --- Draw the underlying graph.
-   [show3d](show3d.ipynb) --- Draw the underlying graph.
-   [sink](sink.ipynb) --- The sink vertex.
-   [smith\_form](smith_form.ipynb) --- The Smith normal form for the
    Laplacian.
-   [solve](solve.ipynb) --- Approximations of the complex affine zeros
    of the sandpile ideal.
-   [stable\_configs](stable_configs.ipynb) --- Generator for all stable
    configurations.
-   [stationary\_density](stationary_density.ipynb) --- The stationary
    density of the sandpile.
-   [superstables](superstables.ipynb) --- The superstable
    configurations.
-   [symmetric\_recurrents](symmetric_recurrents.ipynb) --- The
    symmetric recurrent configurations.
-   [tutte\_polynomial](tutte_polynomial.ipynb) --- The Tutte
    polynomial.
-   [unsaturated\_ideal](unsaturated_ideal.ipynb) --- The unsaturated,
    homogeneous toppling ideal.
-   [version](version.ipynb) --- The version number of Sage Sandpiles.
-   [zero\_config](zero_config.ipynb) --- The all-zero configuration.
-   [zero\_div](zero_div.ipynb) --- The all-zero divisor.

------------------------------------------------------------------------

**Complete descriptions of Sandpile methods.**

---

**all\_k\_config(k)**

The constant configuration with all values set to $k$.

INPUT:

`k` -- integer

OUTPUT:

SandpileConfig

EXAMPLES:

In [ ]:
s = sandpiles.Diamond()
s.all_k_config(7)

{1: 7, 2: 7, 3: 7}

---

**all\_k\_div(k)**

The divisor with all values set to $k$.

INPUT:

`k` -- integer

OUTPUT:

SandpileDivisor

EXAMPLES:

In [ ]:
S = sandpiles.House()
S.all_k_div(7)

{0: 7, 1: 7, 2: 7, 3: 7, 4: 7}

---

**avalanche\_polynomial(multivariable=True)**

The avalanche polynomial. See NOTE for details.

INPUT:

`multivariable` -- (default: `True`) boolean

OUTPUT:

polynomial

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
s.avalanche_polynomial()

9*x0*x1*x2 + 2*x0*x1 + 2*x0*x2 + 2*x1*x2 + 3*x0 + 3*x1 + 3*x2 + 24

In [ ]:
s.avalanche_polynomial(False)

9*x0^3 + 6*x0^2 + 9*x0 + 24

Note

For each nonsink vertex $v$, let $x_v$ be an indeterminate. If $(r,v)$
is a pair consisting of a recurrent $r$ and nonsink vertex $v$, then for
each nonsink vertex $w$, let $n_w$ be the number of times vertex $w$
fires in the stabilization of $r + v$. Let $M(r,v)$ be the monomial
$\prod_w x_w^{n_w}$, i.e., the exponent records the vector of $n_w$ as
$w$ ranges over the nonsink vertices. The avalanche polynomial is then
the sum of $M(r,v)$ as $r$ ranges over the recurrents and $v$ ranges
over the nonsink vertices. If `multivariable` is `False`, then set all
the indeterminates equal to each other (and, thus, only count the number
of vertex firings in the stabilizations, forgetting which particular
vertices fired).

---

**betti(verbose=True)**

The Betti table for the homogeneous toppling ideal. If `verbose` is
`True`, it prints the standard Betti table, otherwise, it returns a less
formatted table.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

Betti numbers for the sandpile

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
S.betti()

           0     1     2     3
------------------------------
    0:     1     -     -     -
    1:     -     2     -     -
    2:     -     4     9     4
------------------------------
total:     1     6     9     4

In [ ]:
S.betti(False)

[1, 6, 9, 4]

---

**betti\_complexes()**

The support-complexes with non-trivial homology. (See NOTE.)

OUTPUT:

list (of pairs \[divisors, corresponding simplicial complex\])

EXAMPLES:

In [ ]:
S = Sandpile({0:{},1:{0: 1, 2: 1, 3: 4},2:{3: 5},3:{1: 1, 2: 1}},0)
p = S.betti_complexes()
p[0]

[{0: -8, 1: 5, 2: 4, 3: 1}, Simplicial complex with vertex set (1, 2, 3) and facets {(3,), (1, 2)}]

In [ ]:
S.resolution()

'R^1 <-- R^5 <-- R^5 <-- R^1'

In [ ]:
S.betti()

           0     1     2     3
------------------------------
    0:     1     -     -     -
    1:     -     5     5     -
    2:     -     -     -     1
------------------------------
total:     1     5     5     1

In [ ]:
len(p)

11

In [ ]:
p[0][1].homology()

{0: Z, 1: 0}

In [ ]:
p[-1][1].homology()

{0: 0, 1: 0, 2: Z}

Note

A `support-complex` is the simplicial complex formed from the supports
of the divisors in a linear system.

---

**burning\_config()**

The minimal burning configuration.

OUTPUT:

dict (configuration)

EXAMPLES:

In [ ]:
g = {0:{},1:{0:1,3:1,4:1},2:{0:1,3:1,5:1}, \

           3:{2:1,5:1},4:{1:1,3:1},5:{2:1,3:1}}

In [ ]:
S = Sandpile(g,0)
S.burning_config()

{1: 2, 2: 0, 3: 1, 4: 1, 5: 0}

In [ ]:
S.burning_config().values()

[2, 0, 1, 1, 0]

In [ ]:
S.burning_script()

{1: 1, 2: 3, 3: 5, 4: 1, 5: 4}

In [ ]:
script = S.burning_script().values()
script

[1, 3, 5, 1, 4]

In [ ]:
matrix(script)*S.reduced_laplacian()

[2 0 1 1 0]

Note

The burning configuration and script are computed using a modified
version of Speer's script algorithm. This is a generalization to
directed multigraphs of Dhar's burning algorithm.

A *burning configuration* is a nonnegative integer-linear combination of
the rows of the reduced Laplacian matrix having nonnegative entries and
such that every vertex has a path from some vertex in its support. The
corresponding *burning script* gives the integer-linear combination
needed to obtain the burning configuration. So if $b$ is the burning
configuration, $\sigma$ is its script, and $\tilde{L}$ is the reduced
Laplacian, then $\sigma\cdot
\tilde{L} = b$. The *minimal burning configuration* is the one with the
minimal script (its components are no larger than the components of any
other script for a burning configuration).

The following are equivalent for a configuration $c$ with burning
configuration $b$ having script $\sigma$ :

> -   $c$ is recurrent;
> -   $c+b$ stabilizes to $c$;
> -   the firing vector for the stabilization of $c+b$ is $\sigma$.

---

**burning\_script()**

A script for the minimal burning configuration.

OUTPUT:

dict

EXAMPLES:

In [ ]:
g = {0:{},1:{0:1,3:1,4:1},2:{0:1,3:1,5:1},\

3:{2:1,5:1},4:{1:1,3:1},5:{2:1,3:1}}

In [ ]:
S = Sandpile(g,0)
S.burning_config()

{1: 2, 2: 0, 3: 1, 4: 1, 5: 0}

In [ ]:
S.burning_config().values()

[2, 0, 1, 1, 0]

In [ ]:
S.burning_script()

{1: 1, 2: 3, 3: 5, 4: 1, 5: 4}

In [ ]:
script = S.burning_script().values()
script

[1, 3, 5, 1, 4]

In [ ]:
matrix(script)*S.reduced_laplacian()

[2 0 1 1 0]

Note

The burning configuration and script are computed using a modified
version of Speer's script algorithm. This is a generalization to
directed multigraphs of Dhar's burning algorithm.

A *burning configuration* is a nonnegative integer-linear combination of
the rows of the reduced Laplacian matrix having nonnegative entries and
such that every vertex has a path from some vertex in its support. The
corresponding *burning script* gives the integer-linear combination
needed to obtain the burning configuration. So if $b$ is the burning
configuration, $s$ is its script, and $L_{\mathrm{red}}$ is the reduced
Laplacian, then $s\cdot
L_{\mathrm{red}}= b$. The *minimal burning configuration* is the one
with the minimal script (its components are no larger than the
components of any other script for a burning configuration).

The following are equivalent for a configuration $c$ with burning
configuration $b$ having script $s$ :

> -   $c$ is recurrent;
> -   $c+b$ stabilizes to $c$;
> -   the firing vector for the stabilization of $c+b$ is $s$.

---

**canonical\_divisor()**

The canonical divisor. This is the divisor with $\deg(v)-2$ grains of
sand on each vertex (not counting loops). Only for undirected graphs.

OUTPUT:

SandpileDivisor

EXAMPLES:

In [ ]:
S = sandpiles.Complete(4)
S.canonical_divisor()

{0: 1, 1: 1, 2: 1, 3: 1}

In [ ]:
s = Sandpile({0:[1,1],1:[0,0,1,1,1]},0)
s.canonical_divisor()  # loops are disregarded

{0: 0, 1: 0}

Warning

The underlying graph must be undirected.

---

**dict()**

A dictionary of dictionaries representing a directed graph.

OUTPUT:

dict

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
S.dict()

{0: {1: 1, 2: 1},
 1: {0: 1, 2: 1, 3: 1},
 2: {0: 1, 1: 1, 3: 1},
 3: {1: 1, 2: 1}}

In [ ]:
S.sink()

0

---

**genus()**

The genus: (\# non-loop edges) - (\# vertices) + 1. Only defined for
undirected graphs.

OUTPUT:

integer

EXAMPLES:

In [ ]:
sandpiles.Complete(4).genus()

3

In [ ]:
sandpiles.Cycle(5).genus()

1

---

**groebner()**

A Groebner basis for the homogeneous toppling ideal. It is computed with
respect to the standard sandpile ordering (see `ring`).

OUTPUT:

Groebner basis

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
S.groebner()

[x3*x2^2 - x1^2*x0, x2^3 - x3*x1*x0, x3*x1^2 - x2^2*x0, x1^3 - x3*x2*x0, x3^2 - x0^2, x2*x1 - x0^2]

---

**group\_gens(verbose=True)**

A minimal list of generators for the sandpile group. If `verbose` is
`False` then the generators are represented as lists of integers.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

list of SandpileConfig (or of lists of integers if `verbose` is `False`)

EXAMPLES:

In [ ]:
s = sandpiles.Cycle(5)
s.group_gens()

[{1: 0, 2: 1, 3: 1, 4: 1}]

In [ ]:
s.group_gens()[0].order()

5

In [ ]:
s = sandpiles.Complete(5)
s.group_gens(False)

[[2, 3, 2, 2], [2, 2, 3, 2], [2, 2, 2, 3]]

In [ ]:
[i.order() for i in s.group_gens()]

[5, 5, 5]

In [ ]:
s.invariant_factors()

[1, 5, 5, 5]

---

**group\_order()**

The size of the sandpile group.

OUTPUT:

integer

EXAMPLES:

In [ ]:
S = sandpiles.House()
S.group_order()

11

---

**h\_vector()**

The number of superstable configurations in each degree. Equivalently,
this is the list of first differences of the Hilbert function of the
(homogeneous) toppling ideal.

OUTPUT:

list of nonnegative integers

EXAMPLES:

In [ ]:
s = sandpiles.Grid(2,2)
s.hilbert_function()

[1, 5, 15, 35, 66, 106, 146, 178, 192]

In [ ]:
s.h_vector()

[1, 4, 10, 20, 31, 40, 40, 32, 14]

---

**help(verbose=True)**

List of Sandpile-specific methods (not inherited from Graph). If
`verbose`, include short descriptions.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

printed string

EXAMPLES:

In [ ]:
Sandpile.help()

For detailed help with any method FOO listed below,
enter "Sandpile.FOO?" or enter "S.FOO?" for any Sandpile S.
<BLANKLINE>
all_k_config             -- The constant configuration with all values set to k.
all_k_div                -- The divisor with all values set to k.
avalanche_polynomial     -- The avalanche polynomial.
betti                    -- The Betti table for the homogeneous toppling ideal.
betti_complexes          -- The support-complexes with non-trivial homology.
...
unsaturated_ideal        -- The unsaturated, homogeneous toppling ideal.
version                  -- The version number of Sage Sandpiles.
zero_config              -- The all-zero configuration.
zero_div                 -- The all-zero divisor.

---

**hilbert\_function()**

The Hilbert function of the homogeneous toppling ideal.

OUTPUT:

list of nonnegative integers

EXAMPLES:

In [ ]:
s = sandpiles.Wheel(5)
s.hilbert_function()

[1, 5, 15, 31, 45]

In [ ]:
s.h_vector()

[1, 4, 10, 16, 14]

---

**ideal(gens=False)**

The saturated homogeneous toppling ideal. If `gens` is `True`, the
generators for the ideal are returned instead.

INPUT:

`gens` -- (default: `False`) boolean

OUTPUT:

ideal or, optionally, the generators of an ideal

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
S.ideal()

Ideal (x2*x1 - x0^2, x3^2 - x0^2, x1^3 - x3*x2*x0, x3*x1^2 - x2^2*x0, x2^3 - x3*x1*x0, x3*x2^2 - x1^2*x0) of Multivariate Polynomial Ring in x3, x2, x1, x0 over Rational Field

In [ ]:
S.ideal(True)

[x2*x1 - x0^2, x3^2 - x0^2, x1^3 - x3*x2*x0, x3*x1^2 - x2^2*x0, x2^3 - x3*x1*x0, x3*x2^2 - x1^2*x0]

In [ ]:
S.ideal().gens()  # another way to get the generators

[x2*x1 - x0^2, x3^2 - x0^2, x1^3 - x3*x2*x0, x3*x1^2 - x2^2*x0, x2^3 - x3*x1*x0, x3*x2^2 - x1^2*x0]

---

**identity(verbose=True)**

The identity configuration. If `verbose` is `False`, the configuration
are converted to a list of integers.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

SandpileConfig or a list of integers If `verbose` is `False`, the
configuration are converted to a list of integers.

EXAMPLES:

In [ ]:
s = sandpiles.Diamond()
s.identity()

{1: 2, 2: 2, 3: 0}

In [ ]:
s.identity(False)

[2, 2, 0]

In [ ]:
s.identity() & s.max_stable() == s.max_stable()

True

---

**in\_degree(v=None)**

The in-degree of a vertex or a list of all in-degrees.

INPUT:

`v` -- (optional) vertex name

OUTPUT:

integer or dict

EXAMPLES:

In [ ]:
s = sandpiles.House()
s.in_degree()

{0: 2, 1: 2, 2: 3, 3: 3, 4: 2}

In [ ]:
s.in_degree(2)

3

---

**invariant\_factors()**

The invariant factors of the sandpile group.

OUTPUT:

list of integers

EXAMPLES:

In [ ]:
s = sandpiles.Grid(2,2)
s.invariant_factors()

[1, 1, 8, 24]

---

**is\_undirected()**

Is the underlying graph undirected? `True` if $(u,v)$ is and edge if and
only if $(v,u)$ is an edge, each edge with the same weight.

OUTPUT:

boolean

EXAMPLES:

In [ ]:
sandpiles.Complete(4).is_undirected()

True

In [ ]:
s = Sandpile({0:[1,2], 1:[0,2], 2:[0]}, 0)
s.is_undirected()

False

---

**jacobian\_representatives(verbose=True)**

Representatives for the elements of the Jacobian group. If `verbose` is
`False`, then lists representing the divisors are returned.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

list of SandpileDivisor (or of lists representing divisors)

EXAMPLES:

For an undirected graph, divisors of the form `s - deg(s)*sink` as `s`
varies over the superstables forms a distinct set of representatives for
the Jacobian group.:

In [ ]:
s = sandpiles.Complete(3)
s.superstables(False)

[[0, 0], [0, 1], [1, 0]]

In [ ]:
s.jacobian_representatives(False)

[[0, 0, 0], [-1, 0, 1], [-1, 1, 0]]

If the graph is directed, the representatives described above may by
equivalent modulo the rowspan of the Laplacian matrix:

In [ ]:
s = Sandpile({0: {1: 1, 2: 2}, 1: {0: 2, 2: 4}, 2: {0: 4, 1: 2}},0)
s.group_order()

28

In [ ]:
s.jacobian_representatives()

[{0: -5, 1: 3, 2: 2}, {0: -4, 1: 3, 2: 1}]

Let $\tau$ be the nonnegative generator of the kernel of the transpose
of the Laplacian, and let $tau_s$ be it sink component, then the
sandpile group is isomorphic to the direct sum of the cyclic group of
order $\tau_s$ and the Jacobian group. In the example above, we have:

In [ ]:
s.laplacian().left_kernel()

Free module of degree 3 and rank 1 over Integer Ring
Echelon basis matrix:
[14  5  8]

Note

The Jacobian group is the set of all divisors of degree zero modulo the
integer rowspan of the Laplacian matrix.

---

**laplacian()**

The Laplacian matrix of the graph. Its *rows* encode the vertex firing
rules.

OUTPUT:

matrix

EXAMPLES:

In [ ]:
G = sandpiles.Diamond()
G.laplacian()

[ 2 -1 -1  0]
[-1  3 -1 -1]
[-1 -1  3 -1]
[ 0 -1 -1  2]

Warning

The function `laplacian_matrix` should be avoided. It returns the
indegree version of the Laplacian.

---

**markov\_chain(state, distrib=None)**

The sandpile Markov chain for configurations or divisors. The chain
starts at `state`. See NOTE for details.

INPUT:

-   `state` -- SandpileConfig, SandpileDivisor, or list representing one
    of these
-   `distrib` -- (optional) list of nonnegative numbers summing to 1
    (representing a prob. dist.)

OUTPUT:

generator for Markov chain (see NOTE)

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
m = s.markov_chain([0,0,0])
next(m)          # random

{1: 0, 2: 0, 3: 0}

In [ ]:
next(m).values() # random

[0, 0, 0]

In [ ]:
next(m).values() # random

[0, 0, 0]

In [ ]:
next(m).values() # random

[0, 0, 0]

In [ ]:
next(m).values() # random

[0, 1, 0]

In [ ]:
next(m).values() # random

[0, 2, 0]

In [ ]:
next(m).values() # random

[0, 2, 1]

In [ ]:
next(m).values() # random

[1, 2, 1]

In [ ]:
next(m).values() # random

[2, 2, 1]

In [ ]:
m = s.markov_chain(s.zero_div(), [0.1,0.1,0.1,0.7])
next(m).values() # random

[0, 0, 0, 1]

In [ ]:
next(m).values() # random

[0, 0, 1, 1]

In [ ]:
next(m).values() # random

[0, 0, 1, 2]

In [ ]:
next(m).values() # random

[1, 1, 2, 0]

In [ ]:
next(m).values() # random

[1, 1, 2, 1]

In [ ]:
next(m).values() # random

[1, 1, 2, 2]

In [ ]:
next(m).values() # random

[1, 1, 2, 3]

In [ ]:
next(m).values() # random

[1, 1, 2, 4]

In [ ]:
next(m).values() # random

[1, 1, 3, 4]

Note

The `closed sandpile Markov chain` has state space consisting of the
configurations on a sandpile. It transitions from a state by choosing a
vertex at random (according to the probability distribution `distrib`),
dropping a grain of sand at that vertex, and stabilizing. If the chosen
vertex is the sink, the chain stays at the current state.

The `open sandpile Markov chain` has state space consisting of the
recurrent elements, i.e., the state space is the sandpile group. It
transitions from the configuration $c$ by choosing a vertex $v$ at
random according to `distrib`. The next state is the stabilization of
$c+v$. If $v$ is the sink vertex, then the stabilization of $c+v$ is
defined to be $c$.

Note that in either case, if `distrib` is specified, its length is equal
to the total number of vertices (including the sink).

REFERENCES:

---

**max\_stable()**

The maximal stable configuration.

OUTPUT:

SandpileConfig (the maximal stable configuration)

EXAMPLES:

In [ ]:
S = sandpiles.House()
S.max_stable()

{1: 1, 2: 2, 3: 2, 4: 1}

---

**max\_stable\_div()**

The maximal stable divisor.

OUTPUT:

SandpileDivisor (the maximal stable divisor)

EXAMPLES:

In [ ]:
s = sandpiles.Diamond()
s.max_stable_div()

{0: 1, 1: 2, 2: 2, 3: 1}

In [ ]:
s.out_degree()

{0: 2, 1: 3, 2: 3, 3: 2}

---

**max\_superstables(verbose=True)**

The maximal superstable configurations. If the underlying graph is
undirected, these are the superstables of highest degree. If `verbose`
is `False`, the configurations are converted to lists of integers.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

tuple of SandpileConfig

EXAMPLES:

In [ ]:
s = sandpiles.Diamond()
s.superstables(False)

[[0, 0, 0],
 [0, 0, 1],
 [1, 0, 1],
 [0, 2, 0],
 [2, 0, 0],
 [0, 1, 1],
 [1, 0, 0],
 [0, 1, 0]]

In [ ]:
s.max_superstables(False)

[[1, 0, 1], [0, 2, 0], [2, 0, 0], [0, 1, 1]]

In [ ]:
s.h_vector()

[1, 3, 4]

---

**min\_recurrents(verbose=True)**

The minimal recurrent elements. If the underlying graph is undirected,
these are the recurrent elements of least degree. If `verbose` is
`False`, the configurations are converted to lists of integers.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

list of SandpileConfig

EXAMPLES:

In [ ]:
s = sandpiles.Diamond()
s.recurrents(False)

[[2, 2, 1],
 [2, 2, 0],
 [1, 2, 0],
 [2, 0, 1],
 [0, 2, 1],
 [2, 1, 0],
 [1, 2, 1],
 [2, 1, 1]]

In [ ]:
s.min_recurrents(False)

[[1, 2, 0], [2, 0, 1], [0, 2, 1], [2, 1, 0]]

In [ ]:
[i.deg() for i in s.recurrents()]

[5, 4, 3, 3, 3, 3, 4, 4]

---

**nonsink\_vertices()**

The nonsink vertices.

OUTPUT:

list of vertices

EXAMPLES:

In [ ]:
s = sandpiles.Grid(2,3)
s.nonsink_vertices()

[(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3)]

---

**nonspecial\_divisors(verbose=True)**

The nonspecial divisors. Only for undirected graphs. (See NOTE.)

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

list (of divisors)

EXAMPLES:

In [ ]:
S = sandpiles.Complete(4)
ns = S.nonspecial_divisors()
D = ns[0]
D.values()

[-1, 0, 1, 2]

In [ ]:
D.deg()

2

In [ ]:
[i.effective_div() for i in ns]

[[], [], [], [], [], []]

Note

The "nonspecial divisors" are those divisors of degree $g-1$ with empty
linear system. The term is only defined for undirected graphs. Here,
$g = |E| - |V| + 1$ is the genus of the graph (not counting loops as
part of $|E|$). If `verbose` is `False`, the divisors are converted to
lists of integers.

Warning

The underlying graph must be undirected.

---

**out\_degree(v=None)**

The out-degree of a vertex or a list of all out-degrees.

INPUT:

`v` - (optional) vertex name

OUTPUT:

integer or dict

EXAMPLES:

In [ ]:
s = sandpiles.House()
s.out_degree()

{0: 2, 1: 2, 2: 3, 3: 3, 4: 2}

In [ ]:
s.out_degree(2)

3

---

**picard\_representatives(d, verbose=True)**

Representatives of the divisor classes of degree $d$ in the Picard
group. (Also see the documentation for `jacobian_representatives`.)

INPUT:

-   `d` -- integer
-   `verbose` -- (default: `True`) boolean

OUTPUT:

list of SandpileDivisors (or lists representing divisors)

EXAMPLES:

In [ ]:
s = sandpiles.Complete(3)
s.superstables(False)

[[0, 0], [0, 1], [1, 0]]

In [ ]:
s.jacobian_representatives(False)

[[0, 0, 0], [-1, 0, 1], [-1, 1, 0]]

In [ ]:
s.picard_representatives(3,False)

[[3, 0, 0], [2, 0, 1], [2, 1, 0]]

---

**points()**

Generators for the multiplicative group of zeros of the sandpile ideal.

OUTPUT:

list of complex numbers

EXAMPLES:

The sandpile group in this example is cyclic, and hence there is a
single generator for the group of solutions.

In [ ]:
S = sandpiles.Complete(4)
S.points()

[[-I, I, 1], [-I, 1, I]]

---

**postulation()**

The postulation number of the toppling ideal. This is the largest weight
of a superstable configuration of the graph.

OUTPUT:

nonnegative integer

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
s.postulation()

3

---

**recurrents(verbose=True)**

The recurrent configurations. If `verbose` is `False`, the
configurations are converted to lists of integers.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

list of recurrent configurations

EXAMPLES:

In [ ]:
r = Sandpile(graphs.HouseXGraph(),0).recurrents()
r[:3]

[{1: 2, 2: 3, 3: 3, 4: 1}, {1: 1, 2: 3, 3: 3, 4: 0}, {1: 1, 2: 3, 3: 3, 4: 1}]

In [ ]:
sandpiles.Complete(4).recurrents(False)

[[2, 2, 2],
 [2, 2, 1],
 [2, 1, 2],
 [1, 2, 2],
 [2, 2, 0],
 [2, 0, 2],
 [0, 2, 2],
 [2, 1, 1],
 [1, 2, 1],
 [1, 1, 2],
 [2, 1, 0],
 [2, 0, 1],
 [1, 2, 0],
 [1, 0, 2],
 [0, 2, 1],
 [0, 1, 2]]

In [ ]:
sandpiles.Cycle(4).recurrents(False)

[[1, 1, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0]]

---

**reduced\_laplacian()**

The reduced Laplacian matrix of the graph.

OUTPUT:

matrix

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
S.laplacian()

[ 2 -1 -1  0]
[-1  3 -1 -1]
[-1 -1  3 -1]
[ 0 -1 -1  2]

In [ ]:
S.reduced_laplacian()

[ 3 -1 -1]
[-1  3 -1]
[-1 -1  2]

Note

This is the Laplacian matrix with the row and column indexed by the sink
vertex removed.

---

**reorder\_vertices()**

A copy of the sandpile with vertex names permuted. After reordering,
vertex $u$ comes before vertex $v$ in the list of vertices if $u$ is
closer to the sink.

OUTPUT:

Sandpile

EXAMPLES:

In [ ]:
S = Sandpile({0:[1], 2:[0,1], 1:[2]})
S.dict()

{0: {1: 1}, 1: {2: 1}, 2: {0: 1, 1: 1}}

In [ ]:
T = S.reorder_vertices()

The vertices 1 and 2 have been swapped:

In [ ]:
T.dict() 

{0: {1: 1}, 1: {0: 1, 2: 1}, 2: {0: 1}}

---

**resolution(verbose=False)**

A minimal free resolution of the homogeneous toppling ideal. If
`verbose` is `True`, then all of the mappings are returned. Otherwise,
the resolution is summarized.

INPUT:

`verbose` -- (default: `False`) boolean

OUTPUT:

free resolution of the toppling ideal

EXAMPLES:

In [ ]:
S = Sandpile({0: {}, 1: {0: 1, 2: 1, 3: 4}, 2: {3: 5}, 3: {1: 1, 2: 1}},0)
S.resolution()  # a Gorenstein sandpile graph

'R^1 <-- R^5 <-- R^5 <-- R^1'

In [ ]:
S.resolution(True)

[
[ x1^2 - x3*x0 x3*x1 - x2*x0  x3^2 - x2*x1  x2*x3 - x0^2  x2^2 - x1*x0],
<BLANKLINE>
[ x3  x2   0  x0   0]  [ x2^2 - x1*x0]
[-x1 -x3  x2   0 -x0]  [-x2*x3 + x0^2]
[ x0  x1   0  x2   0]  [-x3^2 + x2*x1]
[  0   0 -x1 -x3  x2]  [x3*x1 - x2*x0]
[  0   0  x0  x1 -x3], [ x1^2 - x3*x0]
]

In [ ]:
r = S.resolution(True)
r[0]*r[1]

[0 0 0 0 0]

In [ ]:
r[1]*r[2]

[0]
[0]
[0]
[0]
[0]

---

**ring()**

The ring containing the homogeneous toppling ideal.

OUTPUT:

ring

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
S.ring()

Multivariate Polynomial Ring in x3, x2, x1, x0 over Rational Field

In [ ]:
S.ring().gens()

(x3, x2, x1, x0)

Note

The indeterminate `xi` corresponds to the $i$-th vertex as listed my the
method `vertices`. The term-ordering is degrevlex with indeterminates
ordered according to their distance from the sink (larger indeterminates
are further from the sink).

---

**show(**kwds)\*\*

Draw the underlying graph.

INPUT:

`kwds` -- (optional) arguments passed to the show method for Graph or
DiGraph

EXAMPLES:

In [ ]:
S = Sandpile({0:[], 1:[0,3,4], 2:[0,3,5], 3:[2,5], 4:[1,1], 5:[2,4]})
S.show()
S.show(graph_border=True, edge_labels=True)

---

**show3d(**kwds)\*\*

Draw the underlying graph.

INPUT:

`kwds` -- (optional) arguments passed to the show method for Graph or
DiGraph

EXAMPLES:

In [ ]:
S = sandpiles.House()
S.show3d()

---

**sink()**

The sink vertex.

OUTPUT:

sink vertex

EXAMPLES:

In [ ]:
G = sandpiles.House()
G.sink()

0

In [ ]:
H = sandpiles.Grid(2,2)
H.sink()

(0, 0)

In [ ]:
type(H.sink())

<... 'tuple'>

---

**smith\_form()**

The Smith normal form for the Laplacian. In detail: a list of integer
matrices $D, U, V$ such that $ULV = D$ where $L$ is the transpose of the
Laplacian, $D$ is diagonal, and $U$ and $V$ are invertible over the
integers.

OUTPUT:

list of integer matrices

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
D,U,V = s.smith_form()
D

[1 0 0 0]
[0 4 0 0]
[0 0 4 0]
[0 0 0 0]

In [ ]:
U*s.laplacian()*V == D  # laplacian symmetric => transpose not necessary

True

---

**solve()**

Approximations of the complex affine zeros of the sandpile ideal.

OUTPUT:

list of complex numbers

EXAMPLES:

In [ ]:
S = Sandpile({0: {}, 1: {2: 2}, 2: {0: 4, 1: 1}}, 0)
S.solve()

[[-0.707107000000000 + 0.707107000000000*I,
  0.707107000000000 - 0.707107000000000*I],
 [-0.707107000000000 - 0.707107000000000*I,
  0.707107000000000 + 0.707107000000000*I],
 [-I, -I],
 [I, I],
 [0.707107000000000 + 0.707107000000000*I,
  -0.707107000000000 - 0.707107000000000*I],
 [0.707107000000000 - 0.707107000000000*I,
  -0.707107000000000 + 0.707107000000000*I],
 [1, 1],
 [-1, -1]]

In [ ]:
len(_)

8

In [ ]:
S.group_order()

8

Note

The solutions form a multiplicative group isomorphic to the sandpile
group. Generators for this group are given exactly by `points()`.

---

**stable\_configs(smax=None)**

Generator for all stable configurations. If `smax` is provided, then the
generator gives all stable configurations less than or equal to `smax`.
If `smax` does not represent a stable configuration, then each component
of `smax` is replaced by the corresponding component of the maximal
stable configuration.

INPUT:

`smax` -- (optional) SandpileConfig or list representing a
SandpileConfig

OUTPUT:

generator for all stable configurations

EXAMPLES:

In [ ]:
s = sandpiles.Complete(3)
a = s.stable_configs()
next(a)

{1: 0, 2: 0}

In [ ]:
[i.values() for i in a]

[[0, 1], [1, 0], [1, 1]]

In [ ]:
b = s.stable_configs([1,0])
list(b)

[{1: 0, 2: 0}, {1: 1, 2: 0}]

---

**stationary\_density()**

The stationary density of the sandpile.

OUTPUT:

rational number

EXAMPLES:

In [ ]:
s = sandpiles.Complete(3)
s.stationary_density()

10/9

In [ ]:
s = Sandpile(digraphs.DeBruijn(2,2),'00')
s.stationary_density()

9/8

Note

The stationary density of a sandpile is the sum
$\sum_c (\deg(c) + \deg(s))$ where $\deg(s)$ is the degree of the sink
and the sum is over all recurrent configurations.

REFERENCES:

-   <a href="#Levine2014" class="citation">[Levine2014]</a>

---

**superstables(verbose=True)**

The superstable configurations. If `verbose` is `False`, the
configurations are converted to lists of integers. Superstables for
undirected graphs are also known as `G-parking functions`.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

list of SandpileConfig

EXAMPLES:

In [ ]:
sp = Sandpile(graphs.HouseXGraph(),0).superstables()
sp[:3]

[{1: 0, 2: 0, 3: 0, 4: 0}, {1: 1, 2: 0, 3: 0, 4: 1}, {1: 1, 2: 0, 3: 0, 4: 0}]

In [ ]:
sandpiles.Complete(4).superstables(False)

[[0, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [1, 0, 0],
 [0, 0, 2],
 [0, 2, 0],
 [2, 0, 0],
 [0, 1, 1],
 [1, 0, 1],
 [1, 1, 0],
 [0, 1, 2],
 [0, 2, 1],
 [1, 0, 2],
 [1, 2, 0],
 [2, 0, 1],
 [2, 1, 0]]

In [ ]:
sandpiles.Cycle(4).superstables(False)

[[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]]

---

**symmetric\_recurrents(orbits)**

The symmetric recurrent configurations.

INPUT:

`orbits` - list of lists partitioning the vertices

OUTPUT:

list of recurrent configurations

EXAMPLES:

In [ ]:
S = Sandpile({0: {},
             1: {0: 1, 2: 1, 3: 1},
             2: {1: 1, 3: 1, 4: 1},
             3: {1: 1, 2: 1, 4: 1},
             4: {2: 1, 3: 1}})
S.symmetric_recurrents([[1],[2,3],[4]])

[{1: 2, 2: 2, 3: 2, 4: 1}, {1: 2, 2: 2, 3: 2, 4: 0}]

In [ ]:
S.recurrents()

[{1: 2, 2: 2, 3: 2, 4: 1},
 {1: 2, 2: 2, 3: 2, 4: 0},
 {1: 2, 2: 1, 3: 2, 4: 0},
 {1: 2, 2: 2, 3: 0, 4: 1},
 {1: 2, 2: 0, 3: 2, 4: 1},
 {1: 2, 2: 2, 3: 1, 4: 0},
 {1: 2, 2: 1, 3: 2, 4: 1},
 {1: 2, 2: 2, 3: 1, 4: 1}]

Note

The user is responsible for ensuring that the list of orbits comes from
a group of symmetries of the underlying graph.

---

**tutte\_polynomial()**

The Tutte polynomial of the underlying graph. Only defined for
undirected sandpile graphs.

OUTPUT:

polynomial

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
s.tutte_polynomial()

x^3 + y^3 + 3*x^2 + 4*x*y + 3*y^2 + 2*x + 2*y

In [ ]:
s.tutte_polynomial().subs(x=1)

y^3 + 3*y^2 + 6*y + 6

In [ ]:
s.tutte_polynomial().subs(x=1).coefficients() == s.h_vector()

True

---

**unsaturated\_ideal()**

The unsaturated, homogeneous toppling ideal.

OUTPUT:

ideal

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
S.unsaturated_ideal().gens()

[x1^3 - x3*x2*x0, x2^3 - x3*x1*x0, x3^2 - x2*x1]

In [ ]:
S.ideal().gens()

[x2*x1 - x0^2, x3^2 - x0^2, x1^3 - x3*x2*x0, x3*x1^2 - x2^2*x0, x2^3 - x3*x1*x0, x3*x2^2 - x1^2*x0]

---

**version()**

The version number of Sage Sandpiles.

OUTPUT:

string

EXAMPLES:

In [ ]:
Sandpile.version()

Sage Sandpiles Version 2.4

In [ ]:
S = sandpiles.Complete(3)
S.version()

Sage Sandpiles Version 2.4

---

**zero\_config()**

The all-zero configuration.

OUTPUT:

SandpileConfig

EXAMPLES:

In [ ]:
s = sandpiles.Diamond()
s.zero_config()

{1: 0, 2: 0, 3: 0}

---

**zero\_div()**

The all-zero divisor.

OUTPUT:

SandpileDivisor

EXAMPLES:

In [ ]:
S = sandpiles.House()
S.zero_div()

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0}

---

#### SandpileConfig

**Summary of methods.**

-   `+ <+>` --- Addition of configurations.
-   `& <&>` --- The stabilization of the sum.
-   [greater-equal](greater-equal.ipynb) --- `True` if every component
    of `self` is at least that of `other`.
-   [greater](greater.ipynb) --- `True` if every component of `self` is
    at least that of `other` and the two configurations are not equal.
-   `~ <~>` --- The stabilized configuration.
-   [less-equal](less-equal.ipynb) --- `True` if every component of
    `self` is at most that of `other`.
-   [less](less.ipynb) --- `True` if every component of `self` is at
    most that of `other` and the two configurations are not equal.
-   `\* <mul>` --- The recurrent element equivalent to the sum.
-   `^ <pow>` --- Exponentiation for the \*-operator.
-   [-](neg.ipynb) --- The additive inverse of the configuration.
-   [-](-.ipynb) --- Subtraction of configurations.
-   [add\_random](add_random-config.ipynb) --- Add one grain of sand to
    a random vertex.
-   [burst\_size](burst_size-config.ipynb) --- The burst size of the
    configuration with respect to the given vertex.
-   [deg](deg-config.ipynb) --- The degree of the configuration.
-   [dualize](dualize-config.ipynb) --- The difference with the maximal
    stable configuration.
-   [equivalent\_recurrent](equivalent_recurrent-config.ipynb) --- The
    recurrent configuration equivalent to the given configuration.
-   [equivalent\_superstable](equivalent_superstable-config.ipynb) ---
    The equivalent superstable configuration.
-   [fire\_script](fire_script-config.ipynb) --- Fire the given script.
-   [fire\_unstable](fire_unstable-config.ipynb) --- Fire all unstable
    vertices.
-   [fire\_vertex](fire_vertex-config.ipynb) --- Fire the given vertex.
-   [help](help-config.ipynb) --- List of SandpileConfig methods.
-   [is\_recurrent](is_recurrent-config.ipynb) --- Is the configuration
    recurrent?
-   [is\_stable](is_stable-config.ipynb) --- Is the configuration
    stable?
-   [is\_superstable](is_superstable-config.ipynb) --- Is the
    configuration superstable?
-   [is\_symmetric](is_symmetric-config.ipynb) --- Is the configuration
    symmetric?
-   [order](order-config.ipynb) --- The order of the equivalent
    recurrent element.
-   [sandpile](sandpile-config.ipynb) --- The configuration's underlying
    sandpile.
-   [show](show-config.ipynb) --- Show the configuration.
-   [stabilize](stabilize-config.ipynb) --- The stabilized
    configuration.
-   [support](support-config.ipynb) --- The vertices containing sand.
-   [unstable](unstable-config.ipynb) --- The unstable vertices.
-   [values](values-config.ipynb) --- The values of the configuration as
    a list.

------------------------------------------------------------------------

**Complete descriptions of SandpileConfig methods.**

---

**+**

> Addition of configurations.
>
> INPUT:
>
> `other` -- SandpileConfig
>
> OUTPUT:
>
> sum of `self` and `other`
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> c = SandpileConfig(S, [1,2])
> d = SandpileConfig(S, [3,2])
> c + d
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: 4, 2: 4}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**&**

> The stabilization of the sum.
>
> INPUT:
>
> `other` -- SandpileConfig
>
> OUTPUT:
>
> SandpileConfig
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(4)
> c = SandpileConfig(S, [1,0,0])
> c + c  # ordinary addition
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: 2, 2: 0, 3: 0}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c & c  # add and stabilize
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: 0, 2: 1, 3: 0}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c*c  # add and find equivalent recurrent
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: 1, 2: 1, 3: 1}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> ~(c + c) == c & c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**\>=**

> `True` if every component of `self` is at least that of `other`.
>
> INPUT:
>
> `other` -- SandpileConfig
>
> OUTPUT:
>
> boolean
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> c = SandpileConfig(S, [1,2])
> d = SandpileConfig(S, [2,3])
> e = SandpileConfig(S, [2,0])
> c >= c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> d >= c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c >= d
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> e >= c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c >= e
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**\>**

> `True` if every component of `self` is at least that of `other` and
> the two configurations are not equal.
>
> INPUT:
>
> `other` -- SandpileConfig
>
> OUTPUT:
>
> boolean
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> c = SandpileConfig(S, [1,2])
> d = SandpileConfig(S, [1,3])
> c > c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> d > c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c > d
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**\~**

> The stabilized configuration.
>
> OUTPUT:
>
> `SandpileConfig`
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.House()
> c = S.max_stable() + S.identity()
> ~c == c.stabilize()
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**\<=**

> `True` if every component of `self` is at most that of `other`.
>
> INPUT:
>
> `other` -- SandpileConfig
>
> OUTPUT:
>
> boolean
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> c = SandpileConfig(S, [1,2])
> d = SandpileConfig(S, [2,3])
> e = SandpileConfig(S, [2,0])
> c <= c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c <= d
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> d <= c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c <= e
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> e <= c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**\<**

> `True` if every component of `self` is at most that of `other` and the
> two configurations are not equal.
>
> INPUT:
>
> `other` -- SandpileConfig
>
> OUTPUT:
>
> boolean
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> c = SandpileConfig(S, [1,2])
> d = SandpileConfig(S, [2,3])
> c < c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c < d
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> d < c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> S = Sandpile(graphs.CycleGraph(3), 0)
> c = SandpileConfig(S, [1,2])
> d = SandpileConfig(S, [2,3])
> c < c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c < d
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> d < c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**\***

> If `other` is an configuration, the recurrent element equivalent to
> the sum. If `other` is an integer, the sum of configuration with
> itself `other` times.
>
> INPUT:
>
> `other` -- SandpileConfig or Integer
>
> OUTPUT:
>
> SandpileConfig
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(4)
> c = SandpileConfig(S, [1,0,0])
> c + c  # ordinary addition
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: 2, 2: 0, 3: 0}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c & c  # add and stabilize
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: 0, 2: 1, 3: 0}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c*c  # add and find equivalent recurrent
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: 1, 2: 1, 3: 1}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> (c*c).is_recurrent()
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c*(-c) == S.identity()
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: 1, 2: 0, 3: 0}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c*3
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: 3, 2: 0, 3: 0}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**^**

> The recurrent element equivalent to the sum of the configuration with
> itself $k$ times. If $k$ is negative, do the same for the negation of
> the configuration. If $k$ is zero, return the identity of the sandpile
> group.
>
> INPUT:
>
> `k` -- SandpileConfig
>
> OUTPUT:
>
> SandpileConfig
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(4)
> c = SandpileConfig(S, [1,0,0])
> c^3
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: 1, 2: 1, 3: 0}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> (c + c + c) == c^3
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> (c + c + c).equivalent_recurrent() == c^3
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c^(-1)
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: 1, 2: 1, 3: 0}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> c^0 == S.identity()
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**\_**

> The additive inverse of the configuration.
>
> OUTPUT:
>
> SandpileConfig
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> c = SandpileConfig(S, [1,2])
> -c
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: -1, 2: -2}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**-**

> Subtraction of configurations.
>
> INPUT:
>
> `other` -- SandpileConfig
>
> OUTPUT:
>
> sum of `self` and `other`
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> c = SandpileConfig(S, [1,2])
> d = SandpileConfig(S, [3,2])
> c - d
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{1: -2, 2: 0}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**add\_random(distrib=None)**

Add one grain of sand to a random vertex. Optionally, a probability
distribution, `distrib`, may be placed on the vertices or the nonsink
vertices. See NOTE for details.

INPUT:

`distrib` -- (optional) list of nonnegative numbers summing to 1
(representing a prob. dist.)

OUTPUT:

SandpileConfig

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
c = s.zero_config()
c.add_random() # random

{1: 0, 2: 1, 3: 0}

In [ ]:
c

{1: 0, 2: 0, 3: 0}

In [ ]:
c.add_random([0.1,0.1,0.8]) # random

{1: 0, 2: 0, 3: 1}

In [ ]:
c.add_random([0.7,0.1,0.1,0.1]) # random

{1: 0, 2: 0, 3: 0}

We compute the "sizes" of the avalanches caused by adding random grains
of sand to the maximal stable configuration on a grid graph. The
function `stabilize()` returns the firing vector of the stabilization, a
dictionary whose values say how many times each vertex fires in the
stabilization.:

In [ ]:
S = sandpiles.Grid(10,10)
m = S.max_stable()
a = []
for i in range(1000):
    m = m.add_random()
    m, f = m.stabilize(True)
    a.append(sum(f.values()))
p = list_plot([[log(i+1),log(a.count(i))] for i in [0..max(a)] if a.count(i)])
p.axes_labels(['log(N)','log(D(N))'])
t = text("Distribution of avalanche sizes", (2,2), rgbcolor=(1,0,0))
show(p+t,axes_labels=['log(N)','log(D(N))'])

Note

If `distrib` is `None`, then the probability is the uniform probability
on the nonsink vertices. Otherwise, there are two possibilities:

\(i\) the length of `distrib` is equal to the number of vertices, and
`distrib` represents a probability distribution on all of the vertices.
In that case, the sink may be chosen at random, in which case, the
configuration is unchanged.

\(ii\) Otherwise, the length of `distrib` must be equal to the number of
nonsink vertices, and `distrib` represents a probability distribution on
the nonsink vertices.

Warning

If `distrib != None`, the user is responsible for assuring the sum of
its entries is 1 and that its length is equal to the number of sink
vertices or the number of nonsink vertices.

---

**burst\_size(v)**

The burst size of the configuration with respect to the given vertex.

INPUT:

`v` -- vertex

OUTPUT:

integer

EXAMPLES:

In [ ]:
s = sandpiles.Diamond()
[i.burst_size(0) for i in s.recurrents()]

[1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
[i.burst_size(1) for i in s.recurrents()]

[0, 0, 1, 2, 1, 2, 0, 2]

Note

To define `c.burst(v)`, if $v$ is not the sink, let $c'$ be the unique
recurrent for which the stabilization of $c' + v$ is $c$. The burst size
is then the amount of sand that goes into the sink during this
stabilization. If $v$ is the sink, the burst size is defined to be 1.

REFERENCES:

-   <a href="#Levine2014" class="citation">[Levine2014]</a>

---

**deg()**

The degree of the configuration.

OUTPUT:

integer

EXAMPLES:

In [ ]:
S = sandpiles.Complete(3)
c = SandpileConfig(S, [1,2])
c.deg()

3

---

**dualize()**

The difference with the maximal stable configuration.

OUTPUT:

SandpileConfig

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(3)
c = SandpileConfig(S, [1,2])
S.max_stable()

{1: 1, 2: 1}

In [ ]:
c.dualize()

{1: 0, 2: -1}

In [ ]:
S.max_stable() - c == c.dualize()

True

---

**equivalent\_recurrent(with\_firing\_vector=False)**

The recurrent configuration equivalent to the given configuration.
Optionally, return the corresponding firing vector.

INPUT:

`with_firing_vector` -- (default: `False`) boolean

OUTPUT:

SandpileConfig or \[SandpileConfig, firing\_vector\]

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
c = SandpileConfig(S, [0,0,0])
c.equivalent_recurrent() == S.identity()

True

In [ ]:
x = c.equivalent_recurrent(True)
r = vector([x[0][v] for v in S.nonsink_vertices()])
f = vector([x[1][v] for v in S.nonsink_vertices()])
cv = vector(c.values())
r == cv - f*S.reduced_laplacian()

True

Note

Let $L$ be the reduced Laplacian, $c$ the initial configuration, $r$ the
returned configuration, and $f$ the firing vector. Then $r = c - f\cdot
L$.

---

**equivalent\_superstable(with\_firing\_vector=False)**

The equivalent superstable configuration. Optionally, return the
corresponding firing vector.

INPUT:

`with_firing_vector` -- (default: `False`) boolean

OUTPUT:

SandpileConfig or \[SandpileConfig, firing\_vector\]

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
m = S.max_stable()
m.equivalent_superstable().is_superstable()

True

In [ ]:
x = m.equivalent_superstable(True)
s = vector(x[0].values())
f = vector(x[1].values())
mv = vector(m.values())
s == mv - f*S.reduced_laplacian()

True

Note

Let $L$ be the reduced Laplacian, $c$ the initial configuration, $s$ the
returned configuration, and $f$ the firing vector. Then $s = c - f\cdot
L$.

---

**fire\_script(sigma)**

Fire the given script. In other words, fire each vertex the number of
times indicated by `sigma`.

INPUT:

`sigma` -- SandpileConfig or (list or dict representing a
SandpileConfig)

OUTPUT:

SandpileConfig

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(4)
c = SandpileConfig(S, [1,2,3])
c.unstable()

[2, 3]

In [ ]:
c.fire_script(SandpileConfig(S,[0,1,1]))

{1: 2, 2: 1, 3: 2}

In [ ]:
c.fire_script(SandpileConfig(S,[2,0,0])) == c.fire_vertex(1).fire_vertex(1)

True

---

**fire\_unstable()**

Fire all unstable vertices.

OUTPUT:

SandpileConfig

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(4)
c = SandpileConfig(S, [1,2,3])
c.fire_unstable()

{1: 2, 2: 1, 3: 2}

---

**fire\_vertex(v)**

Fire the given vertex.

INPUT:

`v` -- vertex

OUTPUT:

SandpileConfig

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(3)
c = SandpileConfig(S, [1,2])
c.fire_vertex(2)

{1: 2, 2: 0}

---

**help(verbose=True)**

List of SandpileConfig methods. If `verbose`, include short
descriptions.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

printed string

EXAMPLES:

In [ ]:
SandpileConfig.help()

Shortcuts for SandpileConfig operations:
~c    -- stabilize
c & d -- add and stabilize
c * c -- add and find equivalent recurrent
c^k   -- add k times and find equivalent recurrent
         (taking inverse if k is negative)
<BLANKLINE>
For detailed help with any method FOO listed below,
enter "SandpileConfig.FOO?" or enter "c.FOO?" for any SandpileConfig c.
<BLANKLINE>
add_random             -- Add one grain of sand to a random vertex.
burst_size             -- The burst size of the configuration with respect to the given vertex.
deg                    -- The degree of the configuration.
dualize                -- The difference with the maximal stable configuration.
equivalent_recurrent   -- The recurrent configuration equivalent to the given configuration.
equivalent_superstable -- The equivalent superstable configuration.
fire_script            -- Fire the given script.
fire_unstable          -- Fire all unstable vertices.
fire_vertex            -- Fire the given vertex.
help        

---

**is\_recurrent()**

Is the configuration recurrent?

OUTPUT:

boolean

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
S.identity().is_recurrent()

True

In [ ]:
S.zero_config().is_recurrent()

False

---

**is\_stable()**

Is the configuration stable?

OUTPUT:

boolean

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
S.max_stable().is_stable()

True

In [ ]:
(2*S.max_stable()).is_stable()

False

In [ ]:
(S.max_stable() & S.max_stable()).is_stable()

True

---

**is\_superstable()**

Is the configuration superstable?

OUTPUT:

boolean

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
S.zero_config().is_superstable()

True

---

**is\_symmetric(orbits)**

Is the configuration symmetric? Return `True` if the values of the
configuration are constant over the vertices in each sublist of
`orbits`.

INPUT:

> `orbits` -- list of lists of vertices

OUTPUT:

boolean

EXAMPLES:

In [ ]:
S = Sandpile({0: {},
             1: {0: 1, 2: 1, 3: 1},
             2: {1: 1, 3: 1, 4: 1},
             3: {1: 1, 2: 1, 4: 1},
             4: {2: 1, 3: 1}})
c = SandpileConfig(S, [1, 2, 2, 3])
c.is_symmetric([[2,3]])

True

---

**order()**

The order of the equivalent recurrent element.

OUTPUT:

integer

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
c = SandpileConfig(S,[2,0,1])
c.order()

4

In [ ]:
~(c + c + c + c) == S.identity()

True

In [ ]:
c = SandpileConfig(S,[1,1,0])
c.order()

1

In [ ]:
c.is_recurrent()

False

In [ ]:
c.equivalent_recurrent() == S.identity()

True

---

**sandpile()**

The configuration's underlying sandpile.

OUTPUT:

Sandpile

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
c = S.identity()
c.sandpile()

Diamond sandpile graph: 4 vertices, sink = 0

In [ ]:
c.sandpile() == S

True

---

**show(sink=True, colors=True, heights=False, directed=None,**kwds)\*\*

Show the configuration.

INPUT:

-   `sink` -- (default: `True`) whether to show the sink
-   `colors` -- (default: `True`) whether to color-code the amount of
    sand on each vertex
-   `heights` -- (default: `False`) whether to label each vertex with
    the amount of sand
-   `directed` -- (optional) whether to draw directed edges
-   `kwds` -- (optional) arguments passed to the show method for Graph

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
c = S.identity()
c.show()
c.show(directed=False)
c.show(sink=False,colors=False,heights=True)

---

**stabilize(with\_firing\_vector=False)**

The stabilized configuration. Optionally returns the corresponding
firing vector.

INPUT:

`with_firing_vector` -- (default: `False`) boolean

OUTPUT:

`SandpileConfig` or `[SandpileConfig, firing_vector]`

EXAMPLES:

In [ ]:
S = sandpiles.House()
c = 2*S.max_stable()
c._set_stabilize()
'_stabilize' in c.__dict__

True

In [ ]:
S = sandpiles.House()
c = S.max_stable() + S.identity()
c.stabilize(True)

[{1: 1, 2: 2, 3: 2, 4: 1}, {1: 2, 2: 2, 3: 3, 4: 3}]

In [ ]:
S.max_stable() & S.identity() == c.stabilize()

True

In [ ]:
~c == c.stabilize()

True

---

**support()**

The vertices containing sand.

OUTPUT:

list - support of the configuration

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
c = S.identity()
c

{1: 2, 2: 2, 3: 0}

In [ ]:
c.support()

[1, 2]

---

**unstable()**

The unstable vertices.

OUTPUT:

list of vertices

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(4)
c = SandpileConfig(S, [1,2,3])
c.unstable()

[2, 3]

---

**values()**

The values of the configuration as a list. The list is sorted in the
order of the vertices.

OUTPUT:

list of integers

boolean

EXAMPLES:

In [ ]:
S = Sandpile({'a':['c','b'], 'b':['c','a'], 'c':['a']},'a')
c = SandpileConfig(S, {'b':1, 'c':2})
c

{'b': 1, 'c': 2}

In [ ]:
c.values()

[1, 2]

In [ ]:
S.nonsink_vertices()

['b', 'c']

---

#### SandpileDivisor

**Summary of methods.**

-   `+ <+-divisor>` --- Addition of divisors.
-   [greater-equal](greater-equal-divisor.ipynb) --- `True` if every
    component of `self` is at least that of `other`.
-   [greater](greater-divisor.ipynb) --- `True` if every component of
    `self` is at least that of `other` and the two divisors are not
    equal.
-   [less-equal](less-equal-divisor.ipynb) --- `True` if every component
    of `self` is at most that of `other`.
-   [less](less-divisor.ipynb) --- `True` if every component of `self`
    is at most that of `other` and the two divisors are not equal.
-   [-](neg-divisor.ipynb) --- The additive inverse of the divisor.
-   [-](sub-divisor.ipynb) --- Subtraction of divisors.
-   [Dcomplex](Dcomplex-divisor.ipynb) --- The support-complex.
-   [add\_random](add_random-divisor.ipynb) --- Add one grain of sand to
    a random vertex.
-   [betti](betti-divisor.ipynb) --- The Betti numbers for the
    support-complex.
-   [deg](deg-divisor.ipynb) --- The degree of the divisor.
-   [dualize](dualize-divisor.ipynb) --- The difference with the maximal
    stable divisor.
-   [effective\_div](effective_div-divisor.ipynb) --- All linearly
    equivalent effective divisors.
-   [fire\_script](fire_script-divisor.ipynb) --- Fire the given script.
-   [fire\_unstable](fire_unstable-divisor.ipynb) --- Fire all unstable
    vertices.
-   [fire\_vertex](fire_vertex-divisor.ipynb) --- Fire the given vertex.
-   [help](help-divisor.ipynb) --- List of SandpileDivisor methods.
-   [is\_alive](is_alive-divisor.ipynb) --- Is the divisor stabilizable?
-   [is\_linearly\_equivalent](is_linearly_equivalent-divisor.ipynb) ---
    Is the given divisor linearly equivalent?
-   [is\_q\_reduced](is_q_reduced-divisor.ipynb) --- Is the divisor
    q-reduced?
-   [is\_symmetric](is_symmetric-divisor.ipynb) --- Is the divisor
    symmetric?
-   [is\_weierstrass\_pt](is_weierstrass_pt-divisor.ipynb) --- Is the
    given vertex a Weierstrass point?
-   [polytope](polytope-divisor.ipynb) --- The polytope determining the
    complete linear system.
-   [polytope\_integer\_pts](polytope_integer_pts-divisor.ipynb) --- The
    integer points inside divisor's polytope.
-   [q\_reduced](q_reduced-divisor.ipynb) --- The linearly equivalent
    q-reduced divisor.
-   [rank](rank-divisor.ipynb) --- The rank of the divisor.
-   [sandpile](sandpile-divisor.ipynb) --- The divisor's underlying
    sandpile.
-   [show](show-divisor.ipynb) --- Show the divisor.
-   [simulate\_threshold](simulate_threshold-divisor.ipynb) --- The
    first unstabilizable divisor in the closed Markov chain.
-   [stabilize](stabilize-divisor.ipynb) --- The stabilization of the
    divisor.
-   [support](support-divisor.ipynb) --- List of vertices at which the
    divisor is nonzero.
-   [unstable](unstable-divisor.ipynb) --- The unstable vertices.
-   [values](values-divisor.ipynb) --- The values of the divisor as a
    list.
-   [weierstrass\_div](weierstrass_div-divisor.ipynb) --- The
    Weierstrass divisor.
-   [weierstrass\_gap\_seq](weierstrass_gap_seq-divisor.ipynb) --- The
    Weierstrass gap sequence at the given vertex.
-   [weierstrass\_pts](weierstrass_pts-divisor.ipynb) --- The
    Weierstrass points (vertices).
-   [weierstrass\_rank\_seq](weierstrass_rank_seq-divisor.ipynb) --- The
    Weierstrass rank sequence at the given vertex.

------------------------------------------------------------------------

**Complete descriptions of SandpileDivisor methods.**

---

**+**

> Addition of divisors.
>
> INPUT:
>
> `other` -- SandpileDivisor
>
> OUTPUT:
>
> sum of `self` and `other`
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> D = SandpileDivisor(S, [1,2,3])
> E = SandpileDivisor(S, [3,2,1])
> D + E
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{0: 4, 1: 4, 2: 4}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**\>=**

> `True` if every component of `self` is at least that of `other`.
>
> INPUT:
>
> `other` -- SandpileDivisor
>
> OUTPUT:
>
> boolean
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> D = SandpileDivisor(S, [1,2,3])
> E = SandpileDivisor(S, [2,3,4])
> F = SandpileDivisor(S, [2,0,4])
> D >= D
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> E >= D
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> D >= E
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> F >= D
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> D >= F
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**\>**

> `True` if every component of `self` is at least that of `other` and
> the two divisors are not equal.
>
> INPUT:
>
> `other` -- SandpileDivisor
>
> OUTPUT:
>
> boolean
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> D = SandpileDivisor(S, [1,2,3])
> E = SandpileDivisor(S, [1,3,4])
> D > D
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> E > D
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> D > E
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**\<=**

> `True` if every component of `self` is at most that of `other`.
>
> INPUT:
>
> `other` -- SandpileDivisor
>
> OUTPUT:
>
> boolean
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> D = SandpileDivisor(S, [1,2,3])
> E = SandpileDivisor(S, [2,3,4])
> F = SandpileDivisor(S, [2,0,4])
> D <= D
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> D <= E
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> E <= D
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> D <= F
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> F <= D
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**\<**

> `True` if every component of `self` is at most that of `other` and the
> two divisors are not equal.
>
> INPUT:
>
> `other` -- SandpileDivisor
>
> OUTPUT:
>
> boolean
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> D = SandpileDivisor(S, [1,2,3])
> E = SandpileDivisor(S, [2,3,4])
> D < D
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> D < E
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "True"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> E < D
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "False"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**-**

> The additive inverse of the divisor.
>
> OUTPUT:
>
> SandpileDivisor
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> D = SandpileDivisor(S, [1,2,3])
> -D
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{0: -1, 1: -2, 2: -3}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**-**

> Subtraction of divisors.
>
> INPUT:
>
> `other` -- SandpileDivisor
>
> OUTPUT:
>
> Difference of `self` and `other`
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(3)
> D = SandpileDivisor(S, [1,2,3])
> E = SandpileDivisor(S, [3,2,1])
> D - E
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "{0: -2, 1: 0, 2: 2}"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**Dcomplex()**

The support-complex. (See NOTE.)

OUTPUT:

simplicial complex

EXAMPLES:

In [ ]:
S = sandpiles.House()
p = SandpileDivisor(S, [1,2,1,0,0]).Dcomplex()
p.homology()

{0: 0, 1: Z x Z, 2: 0}

In [ ]:
p.f_vector()

[1, 5, 10, 4]

In [ ]:
p.betti()

{0: 1, 1: 2, 2: 0}

Note

The "support-complex" is the simplicial complex determined by the
supports of the linearly equivalent effective divisors.

---

**add\_random(distrib=None)**

Add one grain of sand to a random vertex.

INPUT:

`distrib` -- (optional) list of nonnegative numbers representing a
probability distribution on the vertices

OUTPUT:

SandpileDivisor

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
D = s.zero_div()
D.add_random() # random

{0: 0, 1: 0, 2: 1, 3: 0}

In [ ]:
D.add_random([0.1,0.1,0.1,0.7]) # random

{0: 0, 1: 0, 2: 0, 3: 1}

Warning

If `distrib` is not `None`, the user is responsible for assuring the sum
of its entries is 1.

---

**betti()**

The Betti numbers for the support-complex. (See NOTE.)

OUTPUT:

dictionary of integers

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(3)
D = SandpileDivisor(S, [2,0,1])
D.betti()

{0: 1, 1: 1}

Note

The "support-complex" is the simplicial complex determined by the
supports of the linearly equivalent effective divisors.

---

**deg()**

The degree of the divisor.

OUTPUT:

integer

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(3)
D = SandpileDivisor(S, [1,2,3])
D.deg()

6

---

**dualize()**

The difference with the maximal stable divisor.

OUTPUT:

SandpileDivisor

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(3)
D = SandpileDivisor(S, [1,2,3])
D.dualize()

{0: 0, 1: -1, 2: -2}

In [ ]:
S.max_stable_div() - D == D.dualize()

True

---

**effective\_div(verbose=True, with\_firing\_vectors=False)**

All linearly equivalent effective divisors. If `verbose` is `False`, the
divisors are converted to lists of integers. If `with_firing_vectors` is
`True` then a list of firing vectors is also given, each of which
prescribes the vertices to be fired in order to obtain an effective
divisor.

INPUT:

-   `verbose` -- (default: `True`) boolean
-   `with_firing_vectors` -- (default: `False`) boolean

OUTPUT:

list (of divisors)

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
D = SandpileDivisor(s,[4,2,0,0])
sorted(D.effective_div(), key=str)

[{0: 0, 1: 2, 2: 0, 3: 4},
 {0: 0, 1: 2, 2: 4, 3: 0},
 {0: 0, 1: 6, 2: 0, 3: 0},
 {0: 1, 1: 3, 2: 1, 3: 1},
 {0: 2, 1: 0, 2: 2, 3: 2},
 {0: 4, 1: 2, 2: 0, 3: 0}]

In [ ]:
sorted(D.effective_div(False))

[[0, 2, 0, 4],
 [0, 2, 4, 0],
 [0, 6, 0, 0],
 [1, 3, 1, 1],
 [2, 0, 2, 2],
 [4, 2, 0, 0]]

In [ ]:
sorted(D.effective_div(with_firing_vectors=True), key=str)

[({0: 0, 1: 2, 2: 0, 3: 4}, (0, -1, -1, -2)),
 ({0: 0, 1: 2, 2: 4, 3: 0}, (0, -1, -2, -1)),
 ({0: 0, 1: 6, 2: 0, 3: 0}, (0, -2, -1, -1)),
 ({0: 1, 1: 3, 2: 1, 3: 1}, (0, -1, -1, -1)),
 ({0: 2, 1: 0, 2: 2, 3: 2}, (0, 0, -1, -1)),
 ({0: 4, 1: 2, 2: 0, 3: 0}, (0, 0, 0, 0))]

In [ ]:
a = _[2]
a[0].values()

[0, 6, 0, 0]

In [ ]:
vector(D.values()) - s.laplacian()*a[1]

(0, 6, 0, 0)

In [ ]:
sorted(D.effective_div(False, True))

[([0, 2, 0, 4], (0, -1, -1, -2)),
 ([0, 2, 4, 0], (0, -1, -2, -1)),
 ([0, 6, 0, 0], (0, -2, -1, -1)),
 ([1, 3, 1, 1], (0, -1, -1, -1)),
 ([2, 0, 2, 2], (0, 0, -1, -1)),
 ([4, 2, 0, 0], (0, 0, 0, 0))]

In [ ]:
D = SandpileDivisor(s,[-1,0,0,0])
D.effective_div(False,True)

[]

---

**fire\_script(sigma)**

Fire the given script. In other words, fire each vertex the number of
times indicated by `sigma`.

INPUT:

`sigma` -- SandpileDivisor or (list or dict representing a
SandpileDivisor)

OUTPUT:

SandpileDivisor

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(3)
D = SandpileDivisor(S, [1,2,3])
D.unstable()

[1, 2]

In [ ]:
D.fire_script([0,1,1])

{0: 3, 1: 1, 2: 2}

In [ ]:
D.fire_script(SandpileDivisor(S,[2,0,0])) == D.fire_vertex(0).fire_vertex(0)

True

---

**fire\_unstable()**

Fire all unstable vertices.

OUTPUT:

SandpileDivisor

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(3)
D = SandpileDivisor(S, [1,2,3])
D.fire_unstable()

{0: 3, 1: 1, 2: 2}

---

**fire\_vertex(v)**

Fire the given vertex.

INPUT:

`v` -- vertex

OUTPUT:

SandpileDivisor

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(3)
D = SandpileDivisor(S, [1,2,3])
D.fire_vertex(1)

{0: 2, 1: 0, 2: 4}

---

**help(verbose=True)**

List of SandpileDivisor methods. If `verbose`, include short
descriptions.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

printed string

EXAMPLES:

In [ ]:
SandpileDivisor.help()

For detailed help with any method FOO listed below,
enter "SandpileDivisor.FOO?" or enter "D.FOO?" for any SandpileDivisor D.
<BLANKLINE>
Dcomplex               -- The support-complex.
add_random             -- Add one grain of sand to a random vertex.
betti                  -- The Betti numbers for the support-complex.
deg                    -- The degree of the divisor.
dualize                -- The difference with the maximal stable divisor.
effective_div          -- All linearly equivalent effective divisors.
fire_script            -- Fire the given script.
fire_unstable          -- Fire all unstable vertices.
fire_vertex            -- Fire the given vertex.
help                   -- List of SandpileDivisor methods.
is_alive               -- Is the divisor stabilizable?
is_linearly_equivalent -- Is the given divisor linearly equivalent?
is_q_reduced           -- Is the divisor q-reduced?
is_symmetric           -- Is the divisor symmetric?
is_weierstrass_pt      -- Is the given vert

---

**is\_alive(cycle=False)**

Is the divisor stabilizable? In other words, will the divisor stabilize
under repeated firings of all unstable vertices? Optionally returns the
resulting cycle.

INPUT:

`cycle` -- (default: `False`) boolean

OUTPUT:

boolean or optionally, a list of SandpileDivisors

EXAMPLES:

In [ ]:
S = sandpiles.Complete(4)
D = SandpileDivisor(S, {0: 4, 1: 3, 2: 3, 3: 2})
D.is_alive()

True

In [ ]:
D.is_alive(True)

[{0: 4, 1: 3, 2: 3, 3: 2}, {0: 3, 1: 2, 2: 2, 3: 5}, {0: 1, 1: 4, 2: 4, 3: 3}]

---

**is\_linearly\_equivalent(D, with\_firing\_vector=False)**

Is the given divisor linearly equivalent? Optionally, returns the firing
vector. (See NOTE.)

INPUT:

-   `D` -- SandpileDivisor or list, tuple, etc. representing a divisor
-   `with_firing_vector` -- (default: `False`) boolean

OUTPUT:

boolean or integer vector

EXAMPLES:

In [ ]:
s = sandpiles.Complete(3)
D = SandpileDivisor(s,[2,0,0])
D.is_linearly_equivalent([0,1,1])

True

In [ ]:
D.is_linearly_equivalent([0,1,1],True)

(0, -1, -1)

In [ ]:
v = vector(D.is_linearly_equivalent([0,1,1],True))
vector(D.values()) - s.laplacian()*v

(0, 1, 1)

In [ ]:
D.is_linearly_equivalent([0,0,0])

False

In [ ]:
D.is_linearly_equivalent([0,0,0],True)

()

Note

-   If `with_firing_vector` is `False`, returns either `True` or
    `False`.
-   If `with_firing_vector` is `True` then: (i) if `self` is linearly
    equivalent to $D$, returns a vector $v$ such that
    `self - v*self.laplacian().transpose() = D`. Otherwise, (ii) if
    `self` is not linearly equivalent to $D$, the output is the empty
    vector, `()`.

---

**is\_q\_reduced()**

Is the divisor $q$-reduced? This would mean that $self = c + kq$ where
$c$ is superstable, $k$ is an integer, and $q$ is the sink vertex.

OUTPUT:

boolean

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
D = SandpileDivisor(s,[2,-3,2,0])
D.is_q_reduced()

False

In [ ]:
SandpileDivisor(s,[10,0,1,2]).is_q_reduced()

True

For undirected or, more generally, Eulerian graphs, $q$-reduced divisors
are linearly equivalent if and only if they are equal. The same does not
hold for general directed graphs:

In [ ]:
s = Sandpile({0:[1],1:[1,1]})
D = SandpileDivisor(s,[-1,1])
Z = s.zero_div()
D.is_q_reduced()

True

In [ ]:
Z.is_q_reduced()

True

In [ ]:
D == Z

False

In [ ]:
D.is_linearly_equivalent(Z)

True

---

**is\_symmetric(orbits)**

Is the divisor symmetric? Return `True` if the values of the
configuration are constant over the vertices in each sublist of
`orbits`.

INPUT:

`orbits` -- list of lists of vertices

OUTPUT:

boolean

EXAMPLES:

In [ ]:
S = sandpiles.House()
S.dict()

{0: {1: 1, 2: 1},
 1: {0: 1, 3: 1},
 2: {0: 1, 3: 1, 4: 1},
 3: {1: 1, 2: 1, 4: 1},
 4: {2: 1, 3: 1}}

In [ ]:
D = SandpileDivisor(S, [0,0,1,1,3])
D.is_symmetric([[2,3], [4]])

True

---

**is\_weierstrass\_pt(v='sink')**

Is the given vertex a Weierstrass point?

INPUT:

`v` -- (default: `sink`) vertex

OUTPUT:

boolean

EXAMPLES:

In [ ]:
s = sandpiles.House()
K = s.canonical_divisor()
K.weierstrass_rank_seq()  # sequence at the sink vertex, 0

(1, 0, -1)

In [ ]:
K.is_weierstrass_pt()

False

In [ ]:
K.weierstrass_rank_seq(4)

(1, 0, 0, -1)

In [ ]:
K.is_weierstrass_pt(4)

True

Note

The vertex $v$ is a (generalized) Weierstrass point for divisor $D$ if
the sequence of ranks $r(D - nv)$ for $n = 0, 1, 2, \dots$ is not
$r(D), r(D)-1, \dots, 0, -1, -1, \dots$.

---

**polytope()**

The polytope determining the complete linear system.

OUTPUT:

polytope

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
D = SandpileDivisor(s,[4,2,0,0])
p = D.polytope()
p.inequalities()

(An inequality (-3, 1, 1) x + 2 >= 0,
 An inequality (1, 1, 1) x + 4 >= 0,
 An inequality (1, -3, 1) x + 0 >= 0,
 An inequality (1, 1, -3) x + 0 >= 0)

In [ ]:
D = SandpileDivisor(s,[-1,0,0,0])
D.polytope()

The empty polyhedron in QQ^3

Note

For a divisor $D$, this is the intersection of (i) the polyhedron
determined by the system of inequalities $L^t x \leq D$ where $L^t$ is
the transpose of the Laplacian with (ii) the hyperplane
$x_{\mathrm{sink\_vertex}} = 0$. The polytope is thought of as sitting
in $(n-1)$-dimensional Euclidean space where $n$ is the number of
vertices.

---

**polytope\_integer\_pts()**

The integer points inside divisor's polytope. The polytope referred to
here is the one determining the divisor's complete linear system (see
the documentation for `polytope`).

OUTPUT:

tuple of integer vectors

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
D = SandpileDivisor(s,[4,2,0,0])
sorted(D.polytope_integer_pts())

[(-2, -1, -1),
 (-1, -2, -1),
 (-1, -1, -2),
 (-1, -1, -1),
 (0, -1, -1),
 (0, 0, 0)]

In [ ]:
D = SandpileDivisor(s,[-1,0,0,0])
D.polytope_integer_pts()

()

---

**q\_reduced(verbose=True)**

The linearly equivalent $q$-reduced divisor.

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

SandpileDivisor or list representing SandpileDivisor

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
D = SandpileDivisor(s,[2,-3,2,0])
D.q_reduced()

{0: -2, 1: 1, 2: 2, 3: 0}

In [ ]:
D.q_reduced(False)

[-2, 1, 2, 0]

Note

The divisor $D$ is $qreduced if `D = c + kq$ where $c$ is superstable,
$k$ is an integer, and $q$ is the sink.

---

**rank(with\_witness=False)**

The rank of the divisor. Optionally returns an effective divisor $E$
such that $D - E$ is not winnable (has an empty complete linear system).

INPUT:

`with_witness` -- (default: `False`) boolean

OUTPUT:

integer or (integer, SandpileDivisor)

EXAMPLES:

In [ ]:
S = sandpiles.Complete(4)
D = SandpileDivisor(S,[4,2,0,0])
D.rank()

3

In [ ]:
D.rank(True)

(3, {0: 3, 1: 0, 2: 1, 3: 0})

In [ ]:
E = _[1]
(D - E).rank()

-1

Riemann-Roch theorem::


In [ ]:
D.rank() - (S.canonical_divisor()-D).rank() == D.deg() + 1 - S.genus()

True

Riemann-Roch theorem::


In [ ]:
D.rank() - (S.canonical_divisor()-D).rank() == D.deg() + 1 - S.genus()

True

In [ ]:
S = Sandpile({0:[1,1,1,2],1:[0,0,0,1,1,1,2,2],2:[2,2,1,1,0]},0) # multigraph with loops
D = SandpileDivisor(S,[4,2,0])
D.rank(True)

(2, {0: 1, 1: 1, 2: 1})

In [ ]:
S = Sandpile({0:[1,2], 1:[0,2,2], 2: [0,1]},0) # directed graph
S.is_undirected()

False

In [ ]:
D = SandpileDivisor(S,[0,2,0])
D.effective_div()

[{0: 0, 1: 2, 2: 0}, {0: 2, 1: 0, 2: 0}]

In [ ]:
D.rank(True)

(0, {0: 0, 1: 0, 2: 1})

In [ ]:
E = D.rank(True)[1]
(D - E).effective_div()

[]

Note

The rank of a divisor $D$ is -1 if $D$ is not linearly equivalent to an
effective divisor (i.e., the dollar game represented by $D$ is
unwinnable). Otherwise, the rank of $D$ is the largest integer $r$ such
that $D - E$ is linearly equivalent to an effective divisor for all
effective divisors $E$ with $\deg(E) = r$.

---

**sandpile()**

The divisor's underlying sandpile.

OUTPUT:

Sandpile

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
D = SandpileDivisor(S,[1,-2,0,3])
D.sandpile()

Diamond sandpile graph: 4 vertices, sink = 0

In [ ]:
D.sandpile() == S

True

---

**show(heights=True, directed=None,**kwds)\*\*

Show the divisor.

INPUT:

-   `heights` -- (default: `True`) whether to label each vertex with the
    amount of sand
-   `directed` -- (optional) whether to draw directed edges
-   `kwds` -- (optional) arguments passed to the show method for Graph

EXAMPLES:

In [ ]:
S = sandpiles.Diamond()
D = SandpileDivisor(S,[1,-2,0,2])
D.show(graph_border=True,vertex_size=700,directed=False)

---

**simulate\_threshold(distrib=None)**

The first unstabilizable divisor in the closed Markov chain. (See NOTE.)

INPUT:

`distrib` -- (optional) list of nonnegative numbers representing a
probability distribution on the vertices

OUTPUT:

SandpileDivisor

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
D = s.zero_div()
D.simulate_threshold()  # random

{0: 2, 1: 3, 2: 1, 3: 2}

In [ ]:
n(mean([D.simulate_threshold().deg() for _ in range(10)]))  # random

7.10000000000000

In [ ]:
n(s.stationary_density()*s.num_verts())

6.93750000000000

Note

Starting at `self`, repeatedly choose a vertex and add a grain of sand
to it. Return the first unstabilizable divisor that is reached. Also see
the `markov_chain` method for the underlying sandpile.

---

**stabilize(with\_firing\_vector=False)**

The stabilization of the divisor. If not stabilizable, return an error.

INPUT:

`with_firing_vector` -- (default: `False`) boolean

EXAMPLES:

In [ ]:
s = sandpiles.Complete(4)
D = SandpileDivisor(s,[0,3,0,0])
D.stabilize()

{0: 1, 1: 0, 2: 1, 3: 1}

In [ ]:
D.stabilize(with_firing_vector=True)

[{0: 1, 1: 0, 2: 1, 3: 1}, {0: 0, 1: 1, 2: 0, 3: 0}]

---

**support()**

List of vertices at which the divisor is nonzero.

OUTPUT:

list representing the support of the divisor

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(4)
D = SandpileDivisor(S, [0,0,1,1])
D.support()

[2, 3]

In [ ]:
S.vertices(sort=True)

[0, 1, 2, 3]

---

**unstable()**

The unstable vertices.

OUTPUT:

list of vertices

EXAMPLES:

In [ ]:
S = sandpiles.Cycle(3)
D = SandpileDivisor(S, [1,2,3])
D.unstable()

[1, 2]

---

**values()**

The values of the divisor as a list. The list is sorted in the order of
the vertices.

OUTPUT:

list of integers

boolean

EXAMPLES:

In [ ]:
S = Sandpile({'a':['c','b'], 'b':['c','a'], 'c':['a']},'a')
D = SandpileDivisor(S, {'a':0, 'b':1, 'c':2})
D

{'a': 0, 'b': 1, 'c': 2}

In [ ]:
D.values()

[0, 1, 2]

In [ ]:
S.vertices(sort=True)

['a', 'b', 'c']

---

**weierstrass\_div(verbose=True)**

The Weierstrass divisor. Its value at a vertex is the weight of that
vertex as a Weierstrass point. (See
`SandpileDivisor.weierstrass_gap_seq`.)

INPUT:

`verbose` -- (default: `True`) boolean

OUTPUT:

SandpileDivisor

EXAMPLES:

In [ ]:
s = sandpiles.Diamond()
D = SandpileDivisor(s,[4,2,1,0])
[D.weierstrass_rank_seq(v) for v in s]

[(5, 4, 3, 2, 1, 0, 0, -1),
 (5, 4, 3, 2, 1, 0, -1),
 (5, 4, 3, 2, 1, 0, 0, 0, -1),
 (5, 4, 3, 2, 1, 0, 0, -1)]

In [ ]:
D.weierstrass_div()

{0: 1, 1: 0, 2: 2, 3: 1}

In [ ]:
k5 = sandpiles.Complete(5)
K = k5.canonical_divisor()
K.weierstrass_div()

{0: 9, 1: 9, 2: 9, 3: 9, 4: 9}

---

**weierstrass\_gap\_seq(v='sink', weight=True)**

The Weierstrass gap sequence at the given vertex. If `weight` is `True`,
then also compute the weight of each gap value.

INPUT:

-   `v` -- (default: `sink`) vertex
-   `weight` -- (default: `True`) boolean

OUTPUT:

list or (list of list) of integers

EXAMPLES:

In [ ]:
s = sandpiles.Cycle(4)
D = SandpileDivisor(s,[2,0,0,0])
[D.weierstrass_gap_seq(v,False) for v in s.vertices(sort=True)]

[(1, 3), (1, 2), (1, 3), (1, 2)]

In [ ]:
[D.weierstrass_gap_seq(v) for v in s.vertices(sort=True)]

[((1, 3), 1), ((1, 2), 0), ((1, 3), 1), ((1, 2), 0)]

In [ ]:
D.weierstrass_gap_seq()  # gap sequence at sink vertex, 0

((1, 3), 1)

In [ ]:
D.weierstrass_rank_seq()  # rank sequence at the sink vertex

(1, 0, 0, -1)

Note

The integer $k$ is a Weierstrass gap for the divisor $D$ at vertex $v$
if the rank of $D - (k-1)v$ does not equal the rank of $D - kv$. Let $r$
be the rank of $D$ and let $k_i$ be the $i$-th gap at $v$. The
Weierstrass weight of $v$ for $D$ is the sum of $(k_i - i)$ as $i$
ranges from $1$ to $r + 1$. It measure the difference between the
sequence $r, r - 1, ..., 0, -1, -1, ...$ and the rank sequence
$\mathrm{rank}(D), \mathrm{rank}(D - v), \mathrm{rank}(D - 2v), \dots$

---

**weierstrass\_pts(with\_rank\_seq=False)**

The Weierstrass points (vertices). Optionally, return the corresponding
rank sequences.

INPUT:

`with_rank_seq` -- (default: `False`) boolean

OUTPUT:

tuple of vertices or list of (vertex, rank sequence)

EXAMPLES:

In [ ]:
s = sandpiles.House()
K = s.canonical_divisor()
K.weierstrass_pts()

(4,)

In [ ]:
K.weierstrass_pts(True)

[(4, (1, 0, 0, -1))]

Note

The vertex $v$ is a (generalized) Weierstrass point for divisor $D$ if
the sequence of ranks $r(D - nv)$ for $n = 0, 1, 2, \dots$ is not
$r(D), r(D)-1, \dots, 0, -1, -1, \dots$

---

**weierstrass\_rank\_seq(v='sink')**

The Weierstrass rank sequence at the given vertex. Computes the rank of
the divisor $D - nv$ starting with $n=0$ and ending when the rank is
$-1$.

INPUT:

`v` -- (default: `sink`) vertex

OUTPUT:

tuple of int

EXAMPLES:

In [ ]:
s = sandpiles.House()
K = s.canonical_divisor()
[K.weierstrass_rank_seq(v) for v in s.vertices(sort=True)]

[(1, 0, -1), (1, 0, -1), (1, 0, -1), (1, 0, -1), (1, 0, 0, -1)]

---

#### Other

-   [firing\_graph](firing_graph.ipynb) --- The firing graph.
-   [parallel\_firing\_graph](parallel_firing_graph.ipynb) --- The
    parallel-firing graph.
-   [sandpiles](sandpiles.ipynb) --- Some examples of sandpiles.
-   [wilmes\_algorithm](wilmes_algorithm.ipynb) --- Find matrix with the
    same integer row span as `M` that is the reduced Laplacian of a
    digraph.

------------------------------------------------------------------------

**Complete descriptions of methods.**

**firing\_graph(S, eff)**

> Creates a digraph with divisors as vertices and edges between two
> divisors $D$ and $E$ if firing a single vertex in $D$ gives $E$.
>
> INPUT:
>
> `S` -- Sandpile `eff` -- list of divisors
>
> OUTPUT:
>
> DiGraph
>
> EXAMPLES:
>
> ```{.python .input}
> S = sandpiles.Cycle(6)
> D = SandpileDivisor(S, [1,1,1,1,2,0])
> eff = D.effective_div()
> firing_graph(S,eff).show3d(edge_size=.005,vertex_size=0.01)
> ```

---

**parallel\_firing\_graph(S,eff)**

> Creates a digraph with divisors as vertices and edges between two
> divisors `D` and `E` if firing all unstable vertices in `D` gives `E`.
>
> INPUT:
>
> `S` - Sandpile `eff` - list of divisors
>
> OUTPUT:
>
> DiGraph
>
> EXAMPLES:
>
> ```{.python .input}
> S = Sandpile(graphs.CycleGraph(6),0)
> D = SandpileDivisor(S, [1,1,1,1,2,0])
> eff = D.effective_div()
> parallel_firing_graph(S,eff).show3d(edge_size=.005,vertex_size=0.01)
> ```

---

**sandpiles**

> Some examples of sandpiles.
>
> Here are the available examples; you can also type "sandpiles." and
> hit tab to get a list:
>
> -   "Complete()"
> -   "Cycle()"
> -   "Diamond()"
> -   "Grid()"
> -   "House()"
>
> EXAMPLES:
>
> ```{.python .input}
> s = sandpiles.Complete(4)
> s.invariant_factors()
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "[1, 4, 4]"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> ```{.python .input}
> s.laplacian()
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "[ 3 -1 -1 -1]\n[-1  3 -1 -1]\n[-1 -1  3 -1]\n[-1 -1 -1  3]"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```

---

**wilmes\_algorithm(M)**

> Computes an integer matrix `L` with the same integer row span as `M`
> and such that `L` is the reduced laplacian of a directed multigraph.
>
> INPUT:
>
> `M` - square integer matrix of full rank
>
> OUTPUT:
>
> `L` - integer matrix
>
> EXAMPLES:
>
> ```{.python .input}
> P = matrix([[2,3,-7,-3],[5,2,-5,5],[8,2,5,4],[-5,-9,6,6]])
> wilmes_algorithm(P)
> ```
>
> ```{.json .output}
> [{"data": {"text/plain": "[ 3279   -79 -1599 -1600]\n[   -1  1539  -136 -1402]\n[    0    -1  1650 -1649]\n[    0     0 -1658  1658]"}, "execution_count": 1, "metadata": {}, "output_type": "execute_result"}]
> ```
>
> NOTE:
>
> The algorithm is due to John Wilmes.

### Help

Documentation for each method is available through the Sage online help
system:

In [ ]:
SandpileConfig.fire_vertex?

Base Class:     <class 'instancemethod'>
String Form:    <unbound method SandpileConfig.fire_vertex>
Namespace:      Interactive
File:           /usr/local/sage-4.7/local/lib/python2.6/site-packages/sage/sandpiles/sandpile.py
Definition:     SandpileConfig.fire_vertex(self, v)
Docstring:
       Fire the vertex ``v``.

       INPUT:

       ``v`` - vertex

       OUTPUT:

       SandpileConfig

       EXAMPLES:

          sage: S = Sandpile(graphs.CycleGraph(3), 0)
          sage: c = SandpileConfig(S, [1,2])
          sage: c.fire_vertex(2)
          {1: 2, 2: 0}

Note

An alternative to `SandpileConfig.fire_vertex?` in the preceding code
example would be `c.fire_vertex?`, if `c` is any SandpileConfig.

Enter `Sandpile.help()`, `SandpileConfig.help()`, and
`SandpileDivisor.help()` for lists of available Sandpile-specific
methods.

General Sage documentation can be found at
<http://doc.sagemath.org/html/en/>.

## Contact

Please contact <davidp@reed.edu> with questions, bug reports, and
suggestions for additional features and other improvements.

<span id="BN" class="citation-label">BN</span>  
Matthew Baker, Serguei Norine, [Riemann-Roch and Abel-Jacobi Theory on a
Finite Graph](http://people.math.gatech.edu/~mbaker/papers.html),
Advances in Mathematics 215 (2007), 766--788.

<span id="BTW" class="citation-label">BTW</span>  
Per Bak, Chao Tang and Kurt Wiesenfeld (1987). *Self-organized
criticality: an explanation of 1/f noise*, Physical Review Letters 60:
381--384; `Bak-Tang-Wiesenfeld_sandpile`.

<span id="CRS" class="citation-label">CRS</span>  
Robert Cori, Dominique Rossin, and Bruno Salvy, *Polynomial ideals for
sandpiles and their Gröbner bases*, Theoretical Computer Science,
276 (2002) no. 1--2, 1--15.

<span id="H" class="citation-label">H</span>  
Alexander E. Holroyd, Lionel Levine, Karola Meszaros, Yuval Peres, James
Propp, David B. Wilson, *Chip-Firing and Rotor-Routing on Directed
Graphs*, `0801.3306`. An older version of this paper appears in *In and
out of Equilibrium II*, Eds. V. Sidoravicius, M. E. Vares, in the Series
Progress in Probability, Birkhauser (2008).

<span id="Levine2014" class="citation-label">Levine2014</span>  
Lionel Levine. Threshold state and a conjecture of Poghosyan, Poghosyan,
Priezzhev and Ruelle, Communications in Mathematical Physics.
`1402.3283`

<span id="PPW2013" class="citation-label">PPW2013</span>  
D. Perkinson, J. Perlman, and J. Wilmes. *Primer for the algebraic
geometry of sandpiles*. Tropical and Non-Archimedean Geometry, Contemp.
Math., 605, Amer. Math. Soc., Providence, RI, 2013. `1112.6163`.